In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [10]:
# Load the MNIST dataset
data = pd.read_csv("mnist_train.csv")


# Najlepsze podejście do tej pory

Założenia:
* Warstwa powinna przyjmować dane wejściowe z poprzedniej warstwy i zwracać dane wyjściowe do następnej warstwy
- Musimy też zapamiętać x, net, a dla każdej warstwy
- 

In [ ]:
import pandas as pd
import math
import numpy as np

class SiecNeuronowa:
    def __init__(self):
        self.warstwy = []  # lista wag dla każdej warstwy
        self.stany = []    # przechowuje x, net, a dla każdej warstwy
    
    def dodaj_warstwe(self, rozmiar_wejscia, rozmiar_wyjscia):
        """Dodaje warstwę z losowymi wagami"""
        wagi = self.wygeneruj_wagi(rozmiar_wejscia, rozmiar_wyjscia)
        self.warstwy.append(wagi)
    
    def wygeneruj_wagi(self, rozmiar_wejscia, rozmiar_wyjscia):
        """Generuje wagi z rozkładu N(0, 1/sqrt(rozmiar_wejscia) )"""
        return np.random.normal(0, 1/math.sqrt(rozmiar_wejscia), 
                               (rozmiar_wyjscia, 1 + rozmiar_wejscia))
    
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-np.clip(x, -250, 250)))
    
    def deriv_sigmoid(self, x):
        # Pochodna funkcji aktywacji
        s = self.sigmoid(x)
        return s * (1 - s)
    
    def forward_prop(self, wejscie_dane):
        """Propagacja w przód przez wszystkie warstwy"""
        self.stany = []  # reset stanów
        
        a = wejscie_dane.reshape(-1, 1)  # wektor kolumnowy
        
        for i, W in enumerate(self.warstwy): # Tutaj zmienić na for W in self.warstwy
            # Dodajemy bias
            x = np.vstack([1, a])
            
            # Obliczamy net i a
            net = W @ x
            a = self.sigmoid(net)
            
            # Zapamiętujemy stan
            self.stany.append({'x': x, 'net': net, 'a': a}) # Zwracamy list≤ę słowników. Może łatwiej byłoby liste krotek
        
        return a.flatten()  # zwracamy jako wektor
    
    def back_prop(self, wyjscie_oczekiwane, stala_uczenia=0.01):
        """Propagacja w tył przez wszystkie warstwy"""
        wyjscie_oczekiwane = wyjscie_oczekiwane.reshape(-1, 1)
        
        # Ostatnia warstwa
        a_ost = self.stany[-1]['a']
        net_ost = self.stany[-1]['net']
        x_ost = self.stany[-1]['x']
        
        # δ[L] = (a[L] - y) ⊙ φ'(net[L])
        dL_da = a_ost - wyjscie_oczekiwane
        delta = dL_da * self.deriv_sigmoid(net_ost)
        
        # Aktualizacja wag ostatniej warstwy
        dL_dW = delta @ x_ost.T
        self.warstwy[-1] -= stala_uczenia * dL_dW # Aktualizujemy wagi od razu
        
        # Propagacja przez pozostałe warstwy
        for l in range(len(self.warstwy)-2, -1, -1): # Iterujemy od przedostatniej warstwy do zerowej (cofamy się)
            W_nastepna = self.warstwy[l+1] # Wagi bierzemy o warstwę wyższe
            net_obecne = self.stany[l]['net']
            x_obecne = self.stany[l]['x']
            
            # δ[l] = (W[l+1]ᵀ · δ[l+1]) ⊙ φ'(net[l]) - pomijamy bias

            # Wykonujemy tu dwa działania jednocześnie dL/dX[l-1] = W[l].T @ dleta[l]
            # Usuwając bias z dL/dX[l-1], otrzymamy dL/da[l-1]
            # I liczymy delta[l-1] = dL/da[l-1] * fi'(net[l-1])

            W_nastepna_bez_biasu = W_nastepna[:, 1:] # pomijamy bias
            delta = (W_nastepna_bez_biasu.T @ delta) * self.deriv_sigmoid(net_obecne)
            
            # Aktualizacja wag
            dL_dW = delta @ x_obecne.T
            self.warstwy[l] -= stala_uczenia * dL_dW
            

    
    def fit(self, X, y, epoki=10):
        """Uczenie sieci"""
        for epoka in range(epoki):
            total_loss = 0
            for i in range(len(X)):
                # Forward propagation
                output = self.forward_prop(X[i])
                
                # Obliczanie straty
                loss = 0.5 * np.sum((output - y[i]) ** 2)
                total_loss += loss
                
                # Backward propagation
                self.back_prop(y[i])
            
            if epoka % 10 == 0:
                avg_loss = total_loss / len(X)
                print(f"Epoka {epoka}, Loss: {avg_loss:.4f}")
    
    def predict(self, X):
        """Predykcja"""
        predictions = []
        for i in range(len(X)):
            output = self.forward_prop(X[i])
            predictions.append(output)
        return np.array(predictions)

# Przykład użycia
def test_sieci():
    # Prosta sieć do testowania
    siec = SiecNeuronowa()
    siec.dodaj_warstwe(3, 2)  # 3 wejścia → 2 neurony
    siec.dodaj_warstwe(2, 2)  # 2 wejścia → 2 neurony
    
    # Test forward propagation
    wejscie = np.array([1, 1, 0])
    print("Wejście:", wejscie)
    
    output = siec.forward_prop(wejscie)
    print("Wyjście:", output)
    
    # Test backward propagation
    y_oczekiwane = np.array([1, 0])
    print("Oczekiwane:", y_oczekiwane)
    
    siec.back_prop(y_oczekiwane)
    print("Wagi po aktualizacji:")
    for i, w in enumerate(siec.warstwy):
        print(f"Warstwa {i}:\n{w}")

# Test na danych MNIST
def test_mnist():
    # Przygotowanie danych (uproszczone)
    from sklearn.datasets import fetch_openml
    from sklearn.preprocessing import StandardScaler
    
    print("Ładowanie danych MNIST...")
    mnist = fetch_openml('mnist_784', version=1, as_frame=False)
    X, y = mnist.data, mnist.target.astype(int)
    
    # Normalizacja i podział
    X = X / 255.0
    X_train, y_train = X[:1000], y[:1000]  # mały podzbiór
    
    # Konwersja etykiet na one-hot
    y_train_onehot = np.zeros((len(y_train), 10))
    y_train_onehot[np.arange(len(y_train)), y_train] = 1
    
    # Tworzenie sieci
    siec = SiecNeuronowa()
    siec.dodaj_warstwe(784, 128)
    siec.dodaj_warstwe(128, 10)
    
    print("Rozpoczynam uczenie...")
    siec.fit(X_train, y_train_onehot, epoki=40)
    
    # Testowanie
    correct = 0
    for i in range(100):  # test na 100 przykładach
        output = siec.forward_prop(X_train[i])
        pred = np.argmax(output)
        if pred == y_train[i]:
            correct += 1
    
    accuracy = correct / 100
    print(f"Dokładność: {accuracy:.4f}")

if __name__ == "__main__":
    print("Test prostej sieci:")
    test_sieci()
    
    print("\n" + "="*50)
    print("Test na danych MNIST:")
    test_mnist()

Test prostej sieci:
Wejście: [1 1 0]
Wyjście: [0.5974812  0.46051038]
Oczekiwane: [1 0]
Wagi po aktualizacji:
Warstwa 0:
[[-0.63093114 -0.14279365  0.06030256  0.86744447]
 [-0.35832295  0.16731602 -0.10701603 -0.03344419]]
Warstwa 1:
[[ 0.19973145  0.32543509  0.21003857]
 [-0.28068419  0.29849856  0.05342589]]

Test na danych MNIST:
Ładowanie danych MNIST...
Rozpoczynam uczenie...
Epoka 0, Loss: 0.4525
Epoka 10, Loss: 0.1761
Epoka 20, Loss: 0.1181
Epoka 30, Loss: 0.0949
Dokładność: 0.9400


In [29]:
# Data preprocessing
data = np.array(data)
m, n = data.shape
np.random.shuffle(data)

NameError: name 'data' is not defined

In [13]:
data_dev = data[0:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = X_dev / 255.

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.
_, m_train = X_train.shape

In [14]:
# Initialize parameters
def init_params():
    w1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    w2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return w1, b1, w2, b2

In [15]:
# Forward propagation
def forward_prop(w1, b1, w2, b2, X):
    z1 = w1.dot(X) + b1
    a1 = ReLU(z1)
    z2 = w2.dot(a1) + b2
    a2 = softmax(z2)
    return z1, a1, z2, a2

In [16]:
# Define activation functions
def ReLU(Z):
    return np.maximum(Z, 0)

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A

In [17]:
# Backpropagation
def back_prop(z1, a1, z2, a2, w2, Y, X):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    dZ2 = a2 - one_hot_Y
    dW2 = 1/m * dZ2.dot(a1.T)
    db2 = 1/m * np.sum(dZ2)
    dZ1 = w2.T.dot(dZ2) * (z1 > 0)
    dW1 = 1/m * dZ1.dot(X.T)
    db1 = 1/m * np.sum(dZ1)
    return dW1, db1, dW2, db2

In [18]:
# Parameter updates
def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 -= alpha * dW1
    b1 -= alpha * db1
    W2 -= alpha * dW2
    b2 -= alpha * db2
    return W1, b1, W2, b2

In [19]:
# Gradient descent
def gradient_descent(X, Y, iterations, alpha):
    w1, b1, w2, b2 = init_params()
    for i in range(iterations):
        z1, a1, z2, a2 = forward_prop(w1, b1, w2, b2, X)
        dW1, db1, dW2, db2 = back_prop(z1, a1, z2, a2, w2, Y, X)
        w1, b1, w2, b2 = update_params(w1, b1, w2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0:
            print("Iterations: ", i)
            print("Accuracy: ", get_accuracy(get_predictions(a2), Y))
    return w1, b1, w2, b2

In [20]:
# Making predictions
def make_predictions(X, W1, b1, W2, b2):
    _, _, _, A2 = forward_prop(W1, b1, W2, b2, X)
    predictions = np.argmax(A2, 0)
    return predictions

In [21]:
# Testing and evaluation
def test_prediction(index, W1, b1, W2, b2):
    current_image = X_train[:, index, None]
    prediction = make_predictions(X_train[:, index, None], W1, b1, W2, b2)
    label = Y_train[index]
    print("Prediction: ", prediction)
    print("Label: ", label)
    current_image = current_image.reshape((28, 28)) * 255
    plt.gray()
    plt.imshow(current_image, interpolation='nearest')
    plt.show()

# Calculate accuracy
def get_accuracy(predictions, Y):
    return np.sum(predictions == Y) / Y.size

In [22]:
test_prediction(0, w1, b1, w2, b2)

NameError: name 'w1' is not defined

### Inne podejście

In [1]:
import numpy as np
import pickle
import gzip

class Warstwa:
    def __init__(self, rozmiar_wejscia, rozmiar_wyjscia, learning_rate=0.01):
        # Inicjalizacja wag i biasów
        self.wagi = np.random.randn(rozmiar_wejscia, rozmiar_wyjscia) * 0.1
        self.bias = np.zeros((1, rozmiar_wyjscia))
        self.learning_rate = learning_rate
        
        # Pamiętanie danych do propagacji wstecznej
        self.wejscie = None
        self.wyjscie = None
    
    def forward(self, wejscie):
        """Propagacja w przód"""
        self.wejscie = wejscie  # zapamiętujemy wejście
        # Obliczenia: wejście * wagi + bias
        self.wyjscie = np.dot(wejscie, self.wagi) + self.bias
        return self.wyjscie
    
    def backward(self, gradient):
        """Propagacja w tył"""
        # Gradient względem wag
        gradient_wag = np.dot(self.wejscie.T, gradient)
        # Gradient względem biasów
        gradient_bias = np.sum(gradient, axis=0, keepdims=True)
        # Gradient do poprzedniej warstwy
        gradient_wejscie = np.dot(gradient, self.wagi.T)
        
        # Aktualizacja wag i biasów
        self.wagi -= self.learning_rate * gradient_wag
        self.bias -= self.learning_rate * gradient_bias
        
        return gradient_wejscie

class WarstwaAktywacji:
    def __init__(self, funkcja_aktywacji, pochodna_funkcji):
        self.funkcja_aktywacji = funkcja_aktywacji
        self.pochodna_funkcji = pochodna_funkcji
        self.wejscie = None
        self.wyjscie = None
    
    def forward(self, wejscie):
        self.wejscie = wejscie
        self.wyjscie = self.funkcja_aktywacji(wejscie)
        return self.wyjscie
    
    def backward(self, gradient):
        return gradient * self.pochodna_funkcji(self.wejscie)

class SiecNeuronowa:
    def __init__(self):
        self.warstwy = []
    
    def dodaj_warstwe(self, warstwa):
        self.warstwy.append(warstwa)
    
    def forward(self, X):
        """Przepuszcza dane przez wszystkie warstwy"""
        output = X
        for warstwa in self.warstwy:
            output = warstwa.forward(output)
        return output
    
    def backward(self, gradient):
        """Propaguje gradient przez wszystkie warstwy wstecz"""
        for warstwa in reversed(self.warstwy):
            gradient = warstwa.backward(gradient)
    
    def fit(self, X, y, epoki=10):
        """Uczy sieć na danych"""
        for epoka in range(epoki):
            # Propagacja w przód
            output = self.forward(X)
            
            # Obliczenie błędu (różnica między przewidywaniem a prawdą)
            error = output - y
            
            # Propagacja w tył
            self.backward(error)
            
            if epoka % 10 == 0:
                loss = np.mean(error**2)
                print(f"Epoka {epoka}, Loss: {loss:.4f}")
    
    def predict(self, X):
        """Przewiduje dla nowych danych"""
        return self.forward(X)

In [2]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def pochodna_sigmoid(x):
    return sigmoid(x) * (1 - sigmoid(x))

def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

def pochodna_softmax(x):
    # Dla softmax w połączeniu z cross-entropy, pochodna jest prosta
    return 1  # Uproszczenie dla tego przykładu

In [5]:
def wczytaj_mnist(sciezka='mnist.pkl.gz'):
    """Wczytuje dane MNIST"""
    with gzip.open(sciezka, 'rb') as f:
        train_set, valid_set, test_set = pickle.load(f, encoding='latin1')
    return train_set, valid_set, test_set

def przygotuj_dane():
    """Przygotowuje dane do uczenia"""
    train_set, valid_set, test_set = wczytaj_mnist()
    
    # Dane treningowe
    X_train = train_set[0]
    y_train = train_set[1]
    
    # Konwersja etykiet na one-hot encoding
    y_train_onehot = np.zeros((y_train.size, 10))
    y_train_onehot[np.arange(y_train.size), y_train] = 1
    
    return X_train, y_train_onehot

# Główny program
def main():
    # Wczytaj dane
    X_train, y_train = przygotuj_dane()
    
    # Stwórz sieć
    siec = SiecNeuronowa()
    
    # Dodaj warstwy
    siec.dodaj_warstwe(Warstwa(784, 128, learning_rate=0.01))  # Warstwa ukryta
    siec.dodaj_warstwe(WarstwaAktywacji(sigmoid, pochodna_sigmoid))
    siec.dodaj_warstwe(Warstwa(128, 10, learning_rate=0.01))   # Warstwa wyjściowa
    siec.dodaj_warstwe(WarstwaAktywacji(softmax, pochodna_softmax))
    
    # Trenuj sieć
    print("Rozpoczynam trening...")
    siec.fit(X_train, y_train, epoki=50)
    
    # Testowanie
    _, _, test_set = wczytaj_mnist()
    X_test = test_set[0]
    y_test = test_set[1]
    
    predictions = siec.predict(X_test)
    predicted_labels = np.argmax(predictions, axis=1)
    accuracy = np.mean(predicted_labels == y_test)
    print(f"Dokładność na zbiorze testowym: {accuracy:.4f}")

if __name__ == "__main__":
    main()

Rozpoczynam trening...
Epoka 0, Loss: 0.1008


C:\Users\dejvi\AppData\Local\Temp\ipykernel_23304\90600536.py:2: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


Epoka 10, Loss: 0.1806
Epoka 20, Loss: 0.1806
Epoka 30, Loss: 0.1820
Epoka 40, Loss: 0.1820
Dokładność na zbiorze testowym: 0.1135


In [6]:
# Eksperyment 1: Różne rozmiary pierwszej warstwy
def eksperyment_rozmiary():
    rozmiary = [32, 64, 128, 256]
    for rozmiar in rozmiary:
        print(f"\nTestowanie z warstwą ukrytą {rozmiar} neuronów:")
        siec = SiecNeuronowa()
        siec.dodaj_warstwe(Warstwa(784, rozmiar, 0.01))
        siec.dodaj_warstwe(WarstwaAktywacji(sigmoid, pochodna_sigmoid))
        siec.dodaj_warstwe(Warstwa(rozmiar, 10, 0.01))
        siec.dodaj_warstwe(WarstwaAktywacji(softmax, pochodna_softmax))
        
        siec.fit(X_train, y_train, epoki=20)

# Eksperyment 2: Dodanie trzeciej warstwy
def trzy_warstwy():
    print("\nSieć z trzema warstwami ukrytymi:")
    siec = SiecNeuronowa()
    siec.dodaj_warstwe(Warstwa(784, 128, 0.01))
    siec.dodaj_warstwe(WarstwaAktywacji(sigmoid, pochodna_sigmoid))
    siec.dodaj_warstwe(Warstwa(128, 64, 0.01))    # Dodatkowa warstwa
    siec.dodaj_warstwe(WarstwaAktywacji(sigmoid, pochodna_sigmoid))
    siec.dodaj_warstwe(Warstwa(64, 10, 0.01))
    siec.dodaj_warstwe(WarstwaAktywacji(softmax, pochodna_softmax))
    
    siec.fit(X_train, y_train, epoki=30)

In [8]:
eksperyment_rozmiary()


Testowanie z warstwą ukrytą 32 neuronów:


NameError: name 'X_train' is not defined

In [10]:
import numpy as np
import pickle
import gzip

# 1. KLASY SIECI NEURONOWEJ
class Warstwa:
    def __init__(self, rozmiar_wejscia, rozmiar_wyjscia, learning_rate=0.01):
        self.wagi = np.random.randn(rozmiar_wejscia, rozmiar_wyjscia) * 0.1
        self.bias = np.zeros((1, rozmiar_wyjscia))
        self.learning_rate = learning_rate
        self.wejscie = None
        self.wyjscie = None
    
    def forward(self, wejscie):
        self.wejscie = wejscie
        self.wyjscie = np.dot(wejscie, self.wagi) + self.bias
        return self.wyjscie
    
    def backward(self, gradient):
        gradient_wag = np.dot(self.wejscie.T, gradient)
        gradient_bias = np.sum(gradient, axis=0, keepdims=True)
        gradient_wejscie = np.dot(gradient, self.wagi.T)
        
        self.wagi -= self.learning_rate * gradient_wag
        self.bias -= self.learning_rate * gradient_bias
        
        return gradient_wejscie

class WarstwaAktywacji:
    def __init__(self, funkcja_aktywacji, pochodna_funkcji):
        self.funkcja_aktywacji = funkcja_aktywacji
        self.pochodna_funkcji = pochodna_funkcji
        self.wejscie = None
        self.wyjscie = None
    
    def forward(self, wejscie):
        self.wejscie = wejscie
        self.wyjscie = self.funkcja_aktywacji(wejscie)
        return self.wyjscie
    
    def backward(self, gradient):
        return gradient * self.pochodna_funkcji(self.wejscie)

class SiecNeuronowa:
    def __init__(self):
        self.warstwy = []
    
    def dodaj_warstwe(self, warstwa):
        self.warstwy.append(warstwa)
    
    def forward(self, X):
        output = X
        for warstwa in self.warstwy:
            output = warstwa.forward(output)
        return output
    
    def backward(self, gradient):
        for warstwa in reversed(self.warstwy):
            gradient = warstwa.backward(gradient)
    
    def fit(self, X, y, epoki=10):
        for epoka in range(epoki):
            output = self.forward(X)
            error = output - y
            self.backward(error)
            
            if epoka % 10 == 0:
                loss = np.mean(error**2)
                print(f"Epoka {epoka}, Loss: {loss:.4f}")
    
    def predict(self, X):
        return self.forward(X)

# 2. FUNKCJE AKTYWACJI
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def pochodna_sigmoid(x):
    s = sigmoid(x)
    return s * (1 - s)

def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

def pochodna_softmax(x):
    return 1

# 3. FUNKCJE DO WCZYTANIA DANYCH
def wczytaj_mnist(sciezka='mnist.pkl.gz'):
    """Wczytuje dane MNIST - jeśli nie masz pliku, użyj tymczasowych danych"""
    try:
        with gzip.open(sciezka, 'rb') as f:
            train_set, valid_set, test_set = pickle.load(f, encoding='latin1')
        return train_set, valid_set, test_set
    except FileNotFoundError:
        print("Plik MNIST nie znaleziony. Tworzę tymczasowe dane...")
        return stworz_tymczasowe_dane()

def stworz_tymczasowe_dane():
    """Tworzy proste dane do testowania jeśli nie ma pliku MNIST"""
    # Proste dane treningowe (100 próbek)
    X_train = np.random.rand(100, 784)
    y_train = np.random.randint(0, 10, 100)
    y_train_onehot = np.zeros((100, 10))
    y_train_onehot[np.arange(100), y_train] = 1
    
    # Proste dane testowe (20 próbek)
    X_test = np.random.rand(20, 784)
    y_test = np.random.randint(0, 10, 20)
    
    return (X_train, y_train), (X_test, y_test), (X_test, y_test)

def przygotuj_dane():
    """Przygotowuje dane do uczenia"""
    train_set, valid_set, test_set = wczytaj_mnist()
    
    if len(train_set) == 2:  # Jeśli to nasze tymczasowe dane
        X_train = train_set[0]
        y_train_onehot = train_set[1]
    else:  # Jeśli to prawdziwe dane MNIST
        X_train = train_set[0]
        y_train = train_set[1]
        y_train_onehot = np.zeros((y_train.size, 10))
        y_train_onehot[np.arange(y_train.size), y_train] = 1
    
    return X_train, y_train_onehot

# 4. EKSPERYMENTY
def eksperyment_rozmiary():
    """Eksperyment 1: Różne rozmiary pierwszej warstwy"""
    X_train, y_train = przygotuj_dane()
    rozmiary = [32, 64, 128, 256]
    
    for rozmiar in rozmiary:
        print(f"\n{'='*50}")
        print(f"Testowanie z warstwą ukrytą {rozmiar} neuronów:")
        print(f"{'='*50}")
        
        siec = SiecNeuronowa()
        siec.dodaj_warstwe(Warstwa(784, rozmiar, 0.01))
        siec.dodaj_warstwe(WarstwaAktywacji(sigmoid, pochodna_sigmoid))
        siec.dodaj_warstwe(Warstwa(rozmiar, 10, 0.01))
        siec.dodaj_warstwe(WarstwaAktywacji(softmax, pochodna_softmax))
        
        siec.fit(X_train, y_train, epoki=20)

def trzy_warstwy():
    """Eksperyment 2: Dodanie trzeciej warstwy"""
    X_train, y_train = przygotuj_dane()
    
    print(f"\n{'='*50}")
    print("Sieć z trzema warstwami ukrytymi:")
    print(f"{'='*50}")
    
    siec = SiecNeuronowa()
    siec.dodaj_warstwe(Warstwa(784, 128, 0.01))
    siec.dodaj_warstwe(WarstwaAktywacji(sigmoid, pochodna_sigmoid))
    siec.dodaj_warstwe(Warstwa(128, 64, 0.01))    # Dodatkowa warstwa
    siec.dodaj_warstwe(WarstwaAktywacji(sigmoid, pochodna_sigmoid))
    siec.dodaj_warstwe(Warstwa(64, 10, 0.01))
    siec.dodaj_warstwe(WarstwaAktywacji(softmax, pochodna_softmax))
    
    siec.fit(X_train, y_train, epoki=30)

# 5. GŁÓWNY PROGRAM
def main():
    print("SIECI NEURONOWE - EKSPERYMENTY")
    print("=" * 60)
    
    # Sprawdź czy dane ładują się poprawnie
    X_train, y_train = przygotuj_dane()
    print(f"Wymiary danych treningowych: X: {X_train.shape}, y: {y_train.shape}")
    
    # Wybierz który eksperyment uruchomić
    while True:
        print("\nWybierz eksperyment:")
        print("1 - Różne rozmiary pierwszej warstwy")
        print("2 - Trzy warstwy ukryte")
        print("3 - Oba eksperymenty")
        print("0 - Wyjście")
        
        wybor = input("Twój wybór (0-3): ").strip()
        
        if wybor == '1':
            eksperyment_rozmiary()
        elif wybor == '2':
            trzy_warstwy()
        elif wybor == '3':
            eksperyment_rozmiary()
            trzy_warstwy()
        elif wybor == '0':
            print("Koniec programu.")
            break
        else:
            print("Nieprawidłowy wybór. Spróbuj ponownie.")

# 6. URUCHOMIENIE
if __name__ == "__main__":
    main()

SIECI NEURONOWE - EKSPERYMENTY
Wymiary danych treningowych: X: (50000, 784), y: (50000,)

Wybierz eksperyment:
1 - Różne rozmiary pierwszej warstwy
2 - Trzy warstwy ukryte
3 - Oba eksperymenty
0 - Wyjście

Sieć z trzema warstwami ukrytymi:


ValueError: operands could not be broadcast together with shapes (50000,10) (50000,) 

In [13]:
import numpy as np
import pickle
import gzip

# 1. KLASY SIECI NEURONOWEJ
class Warstwa:
    def __init__(self, rozmiar_wejscia, rozmiar_wyjscia, learning_rate=0.01):
        self.wagi = np.random.randn(rozmiar_wejscia, rozmiar_wyjscia) * 0.1
        self.bias = np.zeros((1, rozmiar_wyjscia))
        self.learning_rate = learning_rate
        self.wejscie = None
        self.wyjscie = None
    
    def forward(self, wejscie):
        self.wejscie = wejscie
        self.wyjscie = np.dot(wejscie, self.wagi) + self.bias
        return self.wyjscie
    
    def backward(self, gradient):
        gradient_wag = np.dot(self.wejscie.T, gradient)
        gradient_bias = np.sum(gradient, axis=0, keepdims=True)
        gradient_wejscie = np.dot(gradient, self.wagi.T)
        
        self.wagi -= self.learning_rate * gradient_wag
        self.bias -= self.learning_rate * gradient_bias
        
        return gradient_wejscie

class WarstwaAktywacji:
    def __init__(self, funkcja_aktywacji, pochodna_funkcji):
        self.funkcja_aktywacji = funkcja_aktywacji
        self.pochodna_funkcji = pochodna_funkcji
        self.wejscie = None
        self.wyjscie = None
    
    def forward(self, wejscie):
        self.wejscie = wejscie
        self.wyjscie = self.funkcja_aktywacji(wejscie)
        return self.wyjscie
    
    def backward(self, gradient):
        return gradient * self.pochodna_funkcji(self.wejscie)

class SiecNeuronowa:
    def __init__(self):
        self.warstwy = []
    
    def dodaj_warstwe(self, warstwa):
        self.warstwy.append(warstwa)
    
    def forward(self, X):
        output = X
        for warstwa in self.warstwy:
            output = warstwa.forward(output)
        return output
    
    def backward(self, gradient):
        for warstwa in reversed(self.warstwy):
            gradient = warstwa.backward(gradient)
    
    def fit(self, X, y, epoki=10):
        for epoka in range(epoki):
            output = self.forward(X)
            error = output - y
            self.backward(error)
            
            if epoka % 10 == 0:
                loss = np.mean(error**2)
                print(f"Epoka {epoka}, Loss: {loss:.4f}")
    
    def predict(self, X):
        return self.forward(X)

# 2. FUNKCJE AKTYWACJI
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def pochodna_sigmoid(x):
    s = sigmoid(x)
    return s * (1 - s)

def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

def pochodna_softmax(x):
    return 1

# 3. FUNKCJE DO WCZYTANIA DANYCH
def wczytaj_mnist(sciezka='mnist.pkl.gz'):
    """Wczytuje dane MNIST"""
    try:
        with gzip.open(sciezka, 'rb') as f:
            train_set, valid_set, test_set = pickle.load(f, encoding='latin1')
        return train_set, valid_set, test_set
    except FileNotFoundError:
        print("Plik MNIST nie znaleziony. Tworzę tymczasowe dane...")
        return stworz_tymczasowe_dane()

def stworz_tymczasowe_dane():
    """Tworzy proste dane do testowania jeśli nie ma pliku MNIST"""
    # Proste dane treningowe (1000 próbek dla szybszego testowania)
    X_train = np.random.rand(1000, 784)
    y_train = np.random.randint(0, 10, 1000)
    y_train_onehot = np.zeros((1000, 10))
    y_train_onehot[np.arange(1000), y_train] = 1
    
    # Proste dane testowe (200 próbek)
    X_test = np.random.rand(200, 784)
    y_test = np.random.randint(0, 10, 200)
    
    return (X_train, y_train), (X_test, y_test), (X_test, y_test)

def przygotuj_dane():
    """Przygotowuje dane do uczenia - ZWRACA JEDNOLICIE one-hot encoding"""
    train_set, valid_set, test_set = wczytaj_mnist()
    
    if len(train_set) == 2:  # Jeśli to nasze tymczasowe dane
        X_train = train_set[0]
        y_train = train_set[1]
        # Konwersja na one-hot
        y_train_onehot = np.zeros((y_train.size, 10))
        y_train_onehot[np.arange(y_train.size), y_train] = 1
    else:  # Jeśli to prawdziwe dane MNIST
        X_train = train_set[0]
        y_train = train_set[1]
        # Konwersja na one-hot
        y_train_onehot = np.zeros((y_train.size, 10))
        y_train_onehot[np.arange(y_train.size), y_train] = 1
    
    return X_train, y_train_onehot

# 4. EKSPERYMENTY - POPRAWIONE WERSJE
def eksperyment_rozmiary():
    """Eksperyment 1: Różne rozmiary pierwszej warstwy"""
    X_train, y_train_onehot = przygotuj_dane()  # Teraz zawsze one-hot
    
    # Użyj mniejszego podzbioru danych dla szybszego testowania
    X_train_subset = X_train[:1000]  # Pierwsze 1000 próbek
    y_train_subset = y_train_onehot[:1000]
    
    print(f"Wymiary podzbioru: X: {X_train_subset.shape}, y: {y_train_subset.shape}")
    
    rozmiary = [32, 64, 128, 256]
    
    for rozmiar in rozmiary:
        print(f"\n{'='*50}")
        print(f"Testowanie z warstwą ukrytą {rozmiar} neuronów:")
        print(f"{'='*50}")
        
        siec = SiecNeuronowa()
        siec.dodaj_warstwe(Warstwa(784, rozmiar, 0.01))
        siec.dodaj_warstwe(WarstwaAktywacji(sigmoid, pochodna_sigmoid))
        siec.dodaj_warstwe(Warstwa(rozmiar, 10, 0.01))
        siec.dodaj_warstwe(WarstwaAktywacji(softmax, pochodna_softmax))
        
        siec.fit(X_train_subset, y_train_subset, epoki=20)

def trzy_warstwy():
    """Eksperyment 2: Dodanie trzeciej warstwy"""
    X_train, y_train_onehot = przygotuj_dane()  # Teraz zawsze one-hot
    
    # Użyj mniejszego podzbioru danych dla szybszego testowania
    X_train_subset = X_train[:1000]  # Pierwsze 1000 próbek
    y_train_subset = y_train_onehot[:1000]
    
    print(f"Wymiary podzbioru: X: {X_train_subset.shape}, y: {y_train_subset.shape}")
    
    print(f"\n{'='*50}")
    print("Sieć z trzema warstwami ukrytymi:")
    print(f"{'='*50}")
    
    siec = SiecNeuronowa()
    siec.dodaj_warstwe(Warstwa(784, 128, 0.01))
    siec.dodaj_warstwe(WarstwaAktywacji(sigmoid, pochodna_sigmoid))
    siec.dodaj_warstwe(Warstwa(128, 64, 0.01))    # Dodatkowa warstwa
    siec.dodaj_warstwe(WarstwaAktywacji(sigmoid, pochodna_sigmoid))
    siec.dodaj_warstwe(Warstwa(64, 10, 0.01))
    siec.dodaj_warstwe(WarstwaAktywacji(softmax, pochodna_softmax))
    
    siec.fit(X_train_subset, y_train_subset, epoki=30)

# 5. DODATKOWA FUNKCJA DO TESTOWANIA DOKŁADNOŚCI
def testuj_dokladnosc(siec, X_test, y_test):
    """Testuje dokładność sieci na danych testowych"""
    predictions = siec.predict(X_test)
    predicted_labels = np.argmax(predictions, axis=1)
    accuracy = np.mean(predicted_labels == y_test)
    return accuracy

# 6. GŁÓWNY PROGRAM
def main():
    print("SIECI NEURONOWE - EKSPERYMENTY")
    print("=" * 60)
    
    # Sprawdź czy dane ładują się poprawnie
    X_train, y_train_onehot = przygotuj_dane()
    print(f"Wymiary danych treningowych: X: {X_train.shape}, y: {y_train_onehot.shape}")
    
    # Wybierz który eksperyment uruchomić
    while True:
        print("\nWybierz eksperyment:")
        print("1 - Różne rozmiary pierwszej warstwy")
        print("2 - Trzy warstwy ukryte")
        print("3 - Oba eksperymenty")
        print("0 - Wyjście")
        
        wybor = input("Twój wybór (0-3): ").strip()
        
        if wybor == '1':
            eksperyment_rozmiary()
        elif wybor == '2':
            trzy_warstwy()
        elif wybor == '3':
            eksperyment_rozmiary()
            trzy_warstwy()
        elif wybor == '0':
            print("Koniec programu.")
            break
        else:
            print("Nieprawidłowy wybór. Spróbuj ponownie.")

# 7. URUCHOMIENIE
if __name__ == "__main__":
    main()

SIECI NEURONOWE - EKSPERYMENTY
Wymiary danych treningowych: X: (50000, 784), y: (50000, 10)

Wybierz eksperyment:
1 - Różne rozmiary pierwszej warstwy
2 - Trzy warstwy ukryte
3 - Oba eksperymenty
0 - Wyjście
Wymiary podzbioru: X: (1000, 784), y: (1000, 10)

Testowanie z warstwą ukrytą 32 neuronów:
Epoka 0, Loss: 0.0917
Epoka 10, Loss: 0.0790

Testowanie z warstwą ukrytą 64 neuronów:
Epoka 0, Loss: 0.0924
Epoka 10, Loss: 0.0966

Testowanie z warstwą ukrytą 128 neuronów:
Epoka 0, Loss: 0.0938
Epoka 10, Loss: 0.0743

Testowanie z warstwą ukrytą 256 neuronów:
Epoka 0, Loss: 0.1021
Epoka 10, Loss: 0.0854

Wybierz eksperyment:
1 - Różne rozmiary pierwszej warstwy
2 - Trzy warstwy ukryte
3 - Oba eksperymenty
0 - Wyjście
Wymiary podzbioru: X: (1000, 784), y: (1000, 10)

Testowanie z warstwą ukrytą 32 neuronów:
Epoka 0, Loss: 0.0903
Epoka 10, Loss: 0.0800

Testowanie z warstwą ukrytą 64 neuronów:
Epoka 0, Loss: 0.0913
Epoka 10, Loss: 0.0809

Testowanie z warstwą ukrytą 128 neuronów:
Epoka 0, Lo

In [15]:
# UPROSZCZONY TEST
def prosty_test():
    """Uproszczony test na małych danych"""
    # Stwórz bardzo małe dane testowe
    X_test = np.random.rand(100, 784)  # 100 próbek
    y_test = np.random.randint(0, 10, 100)
    y_test_onehot = np.zeros((100, 10))
    y_test_onehot[np.arange(100), y_test] = 1
    
    print("Uproszczony test:")
    print(f"X shape: {X_test.shape}, y shape: {y_test_onehot.shape}")
    
    # Prosta sieć
    siec = SiecNeuronowa()
    siec.dodaj_warstwe(Warstwa(784, 64, 0.01))
    siec.dodaj_warstwe(WarstwaAktywacji(sigmoid, pochodna_sigmoid))
    siec.dodaj_warstwe(Warstwa(64, 10, 0.01))
    siec.dodaj_warstwe(WarstwaAktywacji(softmax, pochodna_softmax))
    
    print("Rozpoczynam trening...")
    siec.fit(X_test, y_test_onehot, epoki=10)
    print("Test zakończony sukcesem!")

# W main() dodaj opcję 4 dla prostego testu

In [18]:
import numpy as np
import pickle
import gzip

# 1. KLASY SIECI NEURONOWEJ
class Warstwa:
    def __init__(self, rozmiar_wejscia, rozmiar_wyjscia, learning_rate=0.01):
        self.wagi = np.random.randn(rozmiar_wejscia, rozmiar_wyjscia) * 0.1
        self.bias = np.zeros((1, rozmiar_wyjscia))
        self.learning_rate = learning_rate
        self.wejscie = None
        self.wyjscie = None
    
    def forward(self, wejscie):
        self.wejscie = wejscie
        self.wyjscie = np.dot(wejscie, self.wagi) + self.bias
        return self.wyjscie
    
    def backward(self, gradient):
        gradient_wag = np.dot(self.wejscie.T, gradient)
        gradient_bias = np.sum(gradient, axis=0, keepdims=True)
        gradient_wejscie = np.dot(gradient, self.wagi.T)
        
        self.wagi -= self.learning_rate * gradient_wag
        self.bias -= self.learning_rate * gradient_bias
        
        return gradient_wejscie

class WarstwaAktywacji:
    def __init__(self, funkcja_aktywacji, pochodna_funkcji):
        self.funkcja_aktywacji = funkcja_aktywacji
        self.pochodna_funkcji = pochodna_funkcji
        self.wejscie = None
        self.wyjscie = None
    
    def forward(self, wejscie):
        self.wejscie = wejscie
        self.wyjscie = self.funkcja_aktywacji(wejscie)
        return self.wyjscie
    
    def backward(self, gradient):
        return gradient * self.pochodna_funkcji(self.wejscie)

class SiecNeuronowa:
    def __init__(self):
        self.warstwy = []
    
    def dodaj_warstwe(self, warstwa):
        self.warstwy.append(warstwa)
    
    def forward(self, X):
        output = X
        for warstwa in self.warstwy:
            output = warstwa.forward(output)
        return output
    
    def backward(self, gradient):
        for warstwa in reversed(self.warstwy):
            gradient = warstwa.backward(gradient)
    
    def fit(self, X, y, epoki=10):
        losses = []
        for epoka in range(epoki):
            output = self.forward(X)
            error = output - y
            self.backward(error)
            
            loss = np.mean(error**2)
            losses.append(loss)
            
            if epoka % 10 == 0:
                print(f"Epoka {epoka}, Loss: {loss:.4f}")
        
        return losses
    
    def predict(self, X):
        return self.forward(X)

# 2. FUNKCJE AKTYWACJI
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def pochodna_sigmoid(x):
    s = sigmoid(x)
    return s * (1 - s)

def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

def pochodna_softmax(x):
    return 1

# 3. FUNKCJE DO WCZYTANIA DANYCH
def wczytaj_mnist(sciezka='mnist.pkl.gz'):
    """Wczytuje dane MNIST"""
    try:
        with gzip.open(sciezka, 'rb') as f:
            train_set, valid_set, test_set = pickle.load(f, encoding='latin1')
        return train_set, valid_set, test_set
    except FileNotFoundError:
        print("Plik MNIST nie znaleziony. Tworzę tymczasowe dane...")
        return stworz_tymczasowe_dane()

def stworz_tymczasowe_dane():
    """Tworzy proste dane do testowania jeśli nie ma pliku MNIST"""
    np.random.seed(42)  # Dla powtarzalności
    X_train = np.random.rand(1000, 784)
    y_train = np.random.randint(0, 10, 1000)
    y_train_onehot = np.zeros((1000, 10))
    y_train_onehot[np.arange(1000), y_train] = 1
    
    X_test = np.random.rand(200, 784)
    y_test = np.random.randint(0, 10, 200)
    
    return (X_train, y_train), (X_test, y_test), (X_test, y_test)

def przygotuj_dane():
    """Przygotowuje dane do uczenia"""
    train_set, valid_set, test_set = wczytaj_mnist()
    
    if len(train_set) == 2:
        X_train = train_set[0]
        y_train = train_set[1]
        y_train_onehot = np.zeros((y_train.size, 10))
        y_train_onehot[np.arange(y_train.size), y_train] = 1
    else:
        X_train = train_set[0]
        y_train = train_set[1]
        y_train_onehot = np.zeros((y_train.size, 10))
        y_train_onehot[np.arange(y_train.size), y_train] = 1
    
    return X_train, y_train_onehot

# 4. EKSPERYMENTY - ULEPSZONE
def eksperyment_rozmiary():
    """Eksperyment 1: Różne rozmiary pierwszej warstwy"""
    X_train, y_train_onehot = przygotuj_dane()
    
    # Użyj mniejszego podzbioru dla szybkości
    X_train_subset = X_train[:1000]
    y_train_subset = y_train_onehot[:1000]
    
    print(f"Wymiary podzbioru: X: {X_train_subset.shape}, y: {y_train_subset.shape}")
    
    rozmiary = [32, 64, 128, 256]
    wyniki = {}
    
    for rozmiar in rozmiary:
        print(f"\n{'='*60}")
        print(f"TEST: Warstwa ukryta {rozmiar} neuronów")
        print(f"{'='*60}")
        
        siec = SiecNeuronowa()
        siec.dodaj_warstwe(Warstwa(784, rozmiar, 0.01))
        siec.dodaj_warstwe(WarstwaAktywacji(sigmoid, pochodna_sigmoid))
        siec.dodaj_warstwe(Warstwa(rozmiar, 10, 0.01))
        siec.dodaj_warstwe(WarstwaAktywacji(softmax, pochodna_softmax))
        
        losses = siec.fit(X_train_subset, y_train_subset, epoki=30)
        wyniki[rozmiar] = losses[-1]  # Zapisz finalny loss
        
        print(f"Final Loss dla {rozmiar} neuronów: {losses[-1]:.4f}")
    
    # Podsumowanie
    print(f"\n{'='*60}")
    print("PODSUMOWANIE EKSPERYMENTU 1:")
    print(f"{'='*60}")
    for rozmiar, loss in wyniki.items():
        print(f"Rozmiar {rozmiar}: Final Loss = {loss:.4f}")
    
    return wyniki

def trzy_warstwy():
    """Eksperyment 2: Dodanie trzeciej warstwy"""
    X_train, y_train_onehot = przygotuj_dane()
    
    X_train_subset = X_train[:1000]
    y_train_subset = y_train_onehot[:1000]
    
    print(f"Wymiary podzbioru: X: {X_train_subset.shape}, y: {y_train_subset.shape}")
    
    print(f"\n{'='*60}")
    print("EKSPERYMENT 2: Trzy warstwy ukryte")
    print(f"{'='*60}")
    
    siec = SiecNeuronowa()
    siec.dodaj_warstwe(Warstwa(784, 128, 0.01))
    siec.dodaj_warstwe(WarstwaAktywacji(sigmoid, pochodna_sigmoid))
    siec.dodaj_warstwe(Warstwa(128, 64, 0.01))
    siec.dodaj_warstwe(WarstwaAktywacji(sigmoid, pochodna_sigmoid))
    siec.dodaj_warstwe(Warstwa(64, 10, 0.01))
    siec.dodaj_warstwe(WarstwaAktywacji(softmax, pochodna_softmax))
    
    losses = siec.fit(X_train_subset, y_train_subset, epoki=30)
    print(f"Final Loss dla 3 warstw: {losses[-1]:.4f}")
    
    return losses[-1]

def szybki_test():
    """Szybki test na bardzo małych danych"""
    print(f"\n{'='*60}")
    print("SZYBKI TEST (100 próbek, 10 epok)")
    print(f"{'='*60}")
    
    # Bardzo małe dane
    X_test = np.random.rand(100, 784)
    y_test = np.random.randint(0, 10, 100)
    y_test_onehot = np.zeros((100, 10))
    y_test_onehot[np.arange(100), y_test] = 1
    
    siec = SiecNeuronowa()
    siec.dodaj_warstwe(Warstwa(784, 32, 0.01))
    siec.dodaj_warstwe(WarstwaAktywacji(sigmoid, pochodna_sigmoid))
    siec.dodaj_warstwe(Warstwa(32, 10, 0.01))
    siec.dodaj_warstwe(WarstwaAktywacji(softmax, pochodna_softmax))
    
    siec.fit(X_test, y_test_onehot, epoki=10)
    print("Szybki test zakończony!")

# 5. GŁÓWNY PROGRAM
def main():
    print("SIECI NEURONOWE - EKSPERYMENTY")
    print("=" * 60)
    
    X_train, y_train_onehot = przygotuj_dane()
    print(f"Pełne dane: X: {X_train.shape}, y: {y_train_onehot.shape}")
    
    while True:
        print("\n" + "="*50)
        print("MENU GŁÓWNE - Wybierz eksperyment:")
        print("="*50)
        print("1 - Różne rozmiary pierwszej warstwy")
        print("2 - Trzy warstwy ukryte") 
        print("3 - Oba eksperymenty")
        print("4 - Szybki test (100 próbek)")
        print("0 - Wyjście")
        print("="*50)
        
        try:
            wybor = input("Twój wybór (0-4): ").strip()
            
            if wybor == '1':
                eksperyment_rozmiary()
            elif wybor == '2':
                trzy_warstwy()
            elif wybor == '3':
                print("\n" + "="*60)
                print("URUCHAMIANIE OBIU EKSPERYMENTÓW")
                print("="*60)
                wyniki1 = eksperyment_rozmiary()
                wyniki2 = trzy_warstwy()
            elif wybor == '4':
                szybki_test()
            elif wybor == '0':
                print("\nDziękuję za użycie programu! Do widzenia!")
                break
            else:
                print("Nieprawidłowy wybór. Wybierz 0-4.")
                
            # Potwierdzenie kontynuacji
            if wybor != '0':
                input("\nNaciśnij Enter aby kontynuować...")
                
        except KeyboardInterrupt:
            print("\n\nPrzerwano przez użytkownika. Do widzenia!")
            break
        except Exception as e:
            print(f"\nWystąpił błąd: {e}")
            print("Spróbuj ponownie.")

if __name__ == "__main__":
    main()

SIECI NEURONOWE - EKSPERYMENTY
Pełne dane: X: (50000, 784), y: (50000, 10)

MENU GŁÓWNE - Wybierz eksperyment:
1 - Różne rozmiary pierwszej warstwy
2 - Trzy warstwy ukryte
3 - Oba eksperymenty
4 - Szybki test (100 próbek)
0 - Wyjście

SZYBKI TEST (100 próbek, 10 epok)
Epoka 0, Loss: 0.0895
Szybki test zakończony!

MENU GŁÓWNE - Wybierz eksperyment:
1 - Różne rozmiary pierwszej warstwy
2 - Trzy warstwy ukryte
3 - Oba eksperymenty
4 - Szybki test (100 próbek)
0 - Wyjście
Wymiary podzbioru: X: (1000, 784), y: (1000, 10)

TEST: Warstwa ukryta 32 neuronów
Epoka 0, Loss: 0.0903
Epoka 10, Loss: 0.0853
Epoka 20, Loss: 0.0797
Final Loss dla 32 neuronów: 0.0751

TEST: Warstwa ukryta 64 neuronów
Epoka 0, Loss: 0.0922
Epoka 10, Loss: 0.0847
Epoka 20, Loss: 0.0950
Final Loss dla 64 neuronów: 0.0579

TEST: Warstwa ukryta 128 neuronów
Epoka 0, Loss: 0.0926
Epoka 10, Loss: 0.0884
Epoka 20, Loss: 0.0869
Final Loss dla 128 neuronów: 0.0779

TEST: Warstwa ukryta 256 neuronów
Epoka 0, Loss: 0.0965
Epoka 1

In [21]:
import numpy as np
import pickle
import gzip

# 1. KLASY SIECI NEURONOWEJ
class Warstwa:
    def __init__(self, rozmiar_wejscia, rozmiar_wyjscia, learning_rate=0.01):
        self.wagi = np.random.randn(rozmiar_wejscia, rozmiar_wyjscia) * 0.1
        self.bias = np.zeros((1, rozmiar_wyjscia))
        self.learning_rate = learning_rate
        self.wejscie = None
        self.wyjscie = None
    
    def forward(self, wejscie):
        self.wejscie = wejscie
        self.wyjscie = np.dot(wejscie, self.wagi) + self.bias
        return self.wyjscie
    
    def backward(self, gradient):
        gradient_wag = np.dot(self.wejscie.T, gradient)
        gradient_bias = np.sum(gradient, axis=0, keepdims=True)
        gradient_wejscie = np.dot(gradient, self.wagi.T)
        
        self.wagi -= self.learning_rate * gradient_wag
        self.bias -= self.learning_rate * gradient_bias
        
        return gradient_wejscie

class WarstwaAktywacji:
    def __init__(self, funkcja_aktywacji, pochodna_funkcji):
        self.funkcja_aktywacji = funkcja_aktywacji
        self.pochodna_funkcji = pochodna_funkcji
        self.wejscie = None
        self.wyjscie = None
    
    def forward(self, wejscie):
        self.wejscie = wejscie
        self.wyjscie = self.funkcja_aktywacji(wejscie)
        return self.wyjscie
    
    def backward(self, gradient):
        return gradient * self.pochodna_funkcji(self.wejscie)

class SiecNeuronowa:
    def __init__(self):
        self.warstwy = []
    
    def dodaj_warstwe(self, warstwa):
        self.warstwy.append(warstwa)
    
    def forward(self, X):
        output = X
        for warstwa in self.warstwy:
            output = warstwa.forward(output)
        return output
    
    def backward(self, gradient):
        for warstwa in reversed(self.warstwy):
            gradient = warstwa.backward(gradient)
    
    def fit(self, X, y, X_val=None, y_val=None, epoki=10):
        losses = []
        accuracies = []
        
        for epoka in range(epoki):
            # Propagacja w przód
            output = self.forward(X)
            error = output - y
            
            # Propagacja w tył
            self.backward(error)
            
            # Obliczanie metryk
            loss = np.mean(error**2)
            losses.append(loss)
            
            # Obliczanie dokładności
            predictions = np.argmax(output, axis=1)
            true_labels = np.argmax(y, axis=1)
            accuracy = np.mean(predictions == true_labels)
            accuracies.append(accuracy)
            
            if epoka % 10 == 0:
                val_accuracy = ""
                if X_val is not None and y_val is not None:
                    val_acc = self.accuracy(X_val, np.argmax(y_val, axis=1))
                    val_accuracy = f", Val Accuracy: {val_acc:.4f}"
                print(f"Epoka {epoka:3d}, Loss: {loss:.4f}, Train Accuracy: {accuracy:.4f}{val_accuracy}")
        
        return losses, accuracies
    
    def predict(self, X):
        return self.forward(X)
    
    def accuracy(self, X, y_true):
        """Oblicza dokładność na danych testowych"""
        predictions = self.predict(X)
        predicted_labels = np.argmax(predictions, axis=1)
        return np.mean(predicted_labels == y_true)

# 2. FUNKCJE AKTYWACJI
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def pochodna_sigmoid(x):
    s = sigmoid(x)
    return s * (1 - s)

def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

def pochodna_softmax(x):
    return 1

# 3. FUNKCJE DO WCZYTANIA DANYCH
def wczytaj_mnist(sciezka='mnist.pkl.gz'):
    """Wczytuje dane MNIST"""
    try:
        with gzip.open(sciezka, 'rb') as f:
            train_set, valid_set, test_set = pickle.load(f, encoding='latin1')
        return train_set, valid_set, test_set
    except FileNotFoundError:
        print("Plik MNIST nie znaleziony. Tworzę tymczasowe dane...")
        return stworz_tymczasowe_dane()

def stworz_tymczasowe_dane():
    """Tworzy proste dane do testowania"""
    np.random.seed(42)
    X_train = np.random.rand(1000, 784)
    y_train = np.random.randint(0, 10, 1000)
    y_train_onehot = np.zeros((1000, 10))
    y_train_onehot[np.arange(1000), y_train] = 1
    
    X_test = np.random.rand(200, 784)
    y_test = np.random.randint(0, 10, 200)
    
    return (X_train, y_train), (X_test, y_test), (X_test, y_test)

def przygotuj_dane():
    """Przygotowuje dane do uczenia z podziałem na train/val/test"""
    train_set, valid_set, test_set = wczytaj_mnist()
    
    if len(train_set) == 2:  # Tymczasowe dane
        X_train, y_train = train_set
        X_val, y_val = valid_set
        X_test, y_test = test_set
    else:  # Prawdziwe MNIST
        X_train, y_train = train_set
        X_val, y_val = valid_set  
        X_test, y_test = test_set
    
    # Konwersja na one-hot tylko dla treningu
    y_train_onehot = np.zeros((y_train.size, 10))
    y_train_onehot[np.arange(y_train.size), y_train] = 1
    
    return (X_train, y_train_onehot, y_train), (X_val, y_val), (X_test, y_test)

# 4. EKSPERYMENTY Z DOKŁADNOŚCIĄ
def eksperyment_rozmiary():
    """Eksperyment 1: Różne rozmiary pierwszej warstwy"""
    (X_train, y_train_onehot, y_train), (X_val, y_val), (X_test, y_test) = przygotuj_dane()
    
    # Użyj mniejszego podzbioru dla szybkości
    X_train_subset = X_train[:1000]
    y_train_subset = y_train_onehot[:1000]
    y_train_labels_subset = y_train[:1000]
    
    X_val_subset = X_val[:200]
    y_val_subset = y_val[:200]
    
    print(f"Train: {X_train_subset.shape}, Val: {X_val_subset.shape}")
    
    rozmiary = [32, 64, 128, 256]
    wyniki = {}
    
    for rozmiar in rozmiary:
        print(f"\n{'='*70}")
        print(f"TEST: Warstwa ukryta {rozmiar} neuronów")
        print(f"{'='*70}")
        
        siec = SiecNeuronowa()
        siec.dodaj_warstwe(Warstwa(784, rozmiar, 0.01))
        siec.dodaj_warstwe(WarstwaAktywacji(sigmoid, pochodna_sigmoid))
        siec.dodaj_warstwe(Warstwa(rozmiar, 10, 0.01))
        siec.dodaj_warstwe(WarstwaAktywacji(softmax, pochodna_softmax))
        
        # Trenuj z walidacją
        losses, accuracies = siec.fit(X_train_subset, y_train_subset, 
                                    X_val_subset, y_val_subset, epoki=30)
        
        # Testuj na danych testowych
        test_accuracy = siec.accuracy(X_test[:200], y_test[:200])
        
        wyniki[rozmiar] = {
            'final_loss': losses[-1],
            'final_train_acc': accuracies[-1],
            'test_accuracy': test_accuracy
        }
        
        print(f"⟹ Final - Train Acc: {accuracies[-1]:.4f}, Test Acc: {test_accuracy:.4f}")
    
    # PODSUMOWANIE
    print(f"\n{'='*70}")
    print("PODSUMOWANIE EKSPERYMENTU 1 - RÓŻNE ROZMIARY WARSTWY")
    print(f"{'='*70}")
    print("Rozmiar | Train Acc | Test Acc  | Final Loss")
    print("-" * 50)
    for rozmiar, metryki in wyniki.items():
        print(f"{rozmiar:7d} | {metryki['final_train_acc']:9.4f} | {metryki['test_accuracy']:9.4f} | {metryki['final_loss']:10.4f}")
    
    # Znajdź najlepszy rozmiar
    najlepszy = max(wyniki.items(), key=lambda x: x[1]['test_accuracy'])
    print(f"\n🏆 NAJLEPSZY: {najlepszy[0]} neuronów (Test Acc: {najlepszy[1]['test_accuracy']:.4f})")
    
    return wyniki

def trzy_warstwy():
    """Eksperyment 2: Dodanie trzeciej warstwy"""
    (X_train, y_train_onehot, y_train), (X_val, y_val), (X_test, y_test) = przygotuj_dane()
    
    X_train_subset = X_train[:1000]
    y_train_subset = y_train_onehot[:1000]
    y_train_labels_subset = y_train[:1000]
    
    X_val_subset = X_val[:200]
    y_val_subset = y_val[:200]
    
    print(f"\n{'='*70}")
    print("EKSPERYMENT 2: TRZY WARSTWY UKRYTE")
    print(f"{'='*70}")
    print("Architektura: 784 → 128 → 64 → 10")
    
    siec = SiecNeuronowa()
    siec.dodaj_warstwe(Warstwa(784, 128, 0.01))
    siec.dodaj_warstwe(WarstwaAktywacji(sigmoid, pochodna_sigmoid))
    siec.dodaj_warstwe(Warstwa(128, 64, 0.01))
    siec.dodaj_warstwe(WarstwaAktywacji(sigmoid, pochodna_sigmoid))
    siec.dodaj_warstwe(Warstwa(64, 10, 0.01))
    siec.dodaj_warstwe(WarstwaAktywacji(softmax, pochodna_softmax))
    
    losses, accuracies = siec.fit(X_train_subset, y_train_subset, 
                                X_val_subset, y_val_subset, epoki=30)
    
    test_accuracy = siec.accuracy(X_test[:200], y_test[:200])
    
    print(f"\n⟹ WYNIK: Train Acc: {accuracies[-1]:.4f}, Test Acc: {test_accuracy:.4f}")
    
    return test_accuracy

def pokaz_przyklady(siec, X_test, y_test, ile=5):
    """Pokazuje przykładowe predykcje"""
    print(f"\nPrzykładowe predykcje (pierwszych {ile}):")
    print("True | Pred | ✓/✗")
    print("-" * 15)
    
    predictions = siec.predict(X_test[:ile])
    predicted_labels = np.argmax(predictions, axis=1)
    
    for i in range(ile):
        true_label = y_test[i]
        pred_label = predicted_labels[i]
        correct = "✓" if true_label == pred_label else "✗"
        print(f"{true_label:4} | {pred_label:4} | {correct}")

# 5. GŁÓWNY PROGRAM
def main():
    print("SIECI NEURONOWE - EKSPERYMENTY Z DOKŁADNOŚCIĄ")
    print("=" * 70)
    
    (X_train, y_train_onehot, y_train), (X_val, y_val), (X_test, y_test) = przygotuj_dane()
    print(f"Dane: Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")
    
    while True:
        print("\n" + "="*50)
        print("MENU GŁÓWNE")
        print("="*50)
        print("1 - Różne rozmiary warstwy (z dokładnością)")
        print("2 - Trzy warstwy ukryte (z dokładnością)") 
        print("3 - Oba eksperymenty")
        print("4 - Pokaż przykładowe predykcje")
        print("0 - Wyjście")
        print("="*50)
        
        try:
            wybor = input("Twój wybór (0-4): ").strip()
            
            if wybor == '1':
                eksperyment_rozmiary()
            elif wybor == '2':
                trzy_warstwy()
            elif wybor == '3':
                print("\n" + "="*70)
                print("URUCHAMIANIE OBIU EKSPERYMENTÓW")
                print("="*70)
                wyniki1 = eksperyment_rozmiary()
                wyniki2 = trzy_warstwy()
            elif wybor == '4':
                # Prosta sieć do pokazania przykładów
                siec = SiecNeuronowa()
                siec.dodaj_warstwe(Warstwa(784, 64, 0.01))
                siec.dodaj_warstwe(WarstwaAktywacji(sigmoid, pochodna_sigmoid))
                siec.dodaj_warstwe(Warstwa(64, 10, 0.01))
                siec.dodaj_warstwe(WarstwaAktywacji(softmax, pochodna_softmax))
                
                # Wytrenuj szybko
                X_temp = X_train[:100]
                y_temp = np.zeros((100, 10))
                y_temp[np.arange(100), y_train[:100]] = 1
                siec.fit(X_temp, y_temp, epoki=5)
                
                pokaz_przyklady(siec, X_test, y_test, 10)
            elif wybor == '0':
                print("\nDo widzenia!")
                break
            else:
                print("Nieprawidłowy wybór.")
                
            if wybor != '0':
                input("\nNaciśnij Enter aby kontynuować...")
                
        except KeyboardInterrupt:
            print("\nPrzerwano. Do widzenia!")
            break

if __name__ == "__main__":
    main()

SIECI NEURONOWE - EKSPERYMENTY Z DOKŁADNOŚCIĄ
Dane: Train: (50000, 784), Val: (10000, 784), Test: (10000, 784)

MENU GŁÓWNE
1 - Różne rozmiary warstwy (z dokładnością)
2 - Trzy warstwy ukryte (z dokładnością)
3 - Oba eksperymenty
4 - Pokaż przykładowe predykcje
0 - Wyjście
Train: (1000, 784), Val: (200, 784)

TEST: Warstwa ukryta 32 neuronów


AxisError: axis 1 is out of bounds for array of dimension 1

# TA WERSJA DZIAŁA OK

In [22]:
import numpy as np
import pickle
import gzip

# 1. KLASY SIECI NEURONOWEJ
class Warstwa:
    def __init__(self, rozmiar_wejscia, rozmiar_wyjscia, learning_rate=0.01):
        self.wagi = np.random.randn(rozmiar_wejscia, rozmiar_wyjscia) * 0.1
        self.bias = np.zeros((1, rozmiar_wyjscia))
        self.learning_rate = learning_rate
        self.wejscie = None
        self.wyjscie = None
    
    def forward(self, wejscie):
        self.wejscie = wejscie
        self.wyjscie = np.dot(wejscie, self.wagi) + self.bias
        return self.wyjscie
    
    def backward(self, gradient):
        gradient_wag = np.dot(self.wejscie.T, gradient)
        gradient_bias = np.sum(gradient, axis=0, keepdims=True)
        gradient_wejscie = np.dot(gradient, self.wagi.T)
        
        self.wagi -= self.learning_rate * gradient_wag
        self.bias -= self.learning_rate * gradient_bias
        
        return gradient_wejscie

class WarstwaAktywacji:
    def __init__(self, funkcja_aktywacji, pochodna_funkcji):
        self.funkcja_aktywacji = funkcja_aktywacji
        self.pochodna_funkcji = pochodna_funkcji
        self.wejscie = None
        self.wyjscie = None
    
    def forward(self, wejscie):
        self.wejscie = wejscie
        self.wyjscie = self.funkcja_aktywacji(wejscie)
        return self.wyjscie
    
    def backward(self, gradient):
        return gradient * self.pochodna_funkcji(self.wejscie)

class SiecNeuronowa:
    def __init__(self):
        self.warstwy = []
    
    def dodaj_warstwe(self, warstwa):
        self.warstwy.append(warstwa)
    
    def forward(self, X):
        output = X
        for warstwa in self.warstwy:
            output = warstwa.forward(output)
        return output
    
    def backward(self, gradient):
        for warstwa in reversed(self.warstwy):
            gradient = warstwa.backward(gradient)
    
    def fit(self, X, y, X_val=None, y_val=None, epoki=10):
        losses = []
        accuracies = []
        
        for epoka in range(epoki):
            # Propagacja w przód
            output = self.forward(X)
            error = output - y
            
            # Propagacja w tył
            self.backward(error)
            
            # Obliczanie metryk
            loss = np.mean(error**2)
            losses.append(loss)
            
            # Obliczanie dokładności
            predictions = np.argmax(output, axis=1)
            true_labels = np.argmax(y, axis=1)
            accuracy = np.mean(predictions == true_labels)
            accuracies.append(accuracy)
            
            if epoka % 10 == 0:
                val_accuracy = ""
                if X_val is not None and y_val is not None:
                    val_acc = self.accuracy(X_val, y_val)  # POPRAWIONE: używamy y_val bez argmax
                    val_accuracy = f", Val Accuracy: {val_acc:.4f}"
                print(f"Epoka {epoka:3d}, Loss: {loss:.4f}, Train Accuracy: {accuracy:.4f}{val_accuracy}")
        
        return losses, accuracies
    
    def predict(self, X):
        return self.forward(X)
    
    def accuracy(self, X, y_true):
        """Oblicza dokładność na danych testowych - y_true to już etykiety, nie one-hot"""
        predictions = self.predict(X)
        predicted_labels = np.argmax(predictions, axis=1)
        return np.mean(predicted_labels == y_true)

# 2. FUNKCJE AKTYWACJI
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def pochodna_sigmoid(x):
    s = sigmoid(x)
    return s * (1 - s)

def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

def pochodna_softmax(x):
    return 1

# 3. FUNKCJE DO WCZYTANIA DANYCH
def wczytaj_mnist(sciezka='mnist.pkl.gz'):
    """Wczytuje dane MNIST"""
    try:
        with gzip.open(sciezka, 'rb') as f:
            train_set, valid_set, test_set = pickle.load(f, encoding='latin1')
        return train_set, valid_set, test_set
    except FileNotFoundError:
        print("Plik MNIST nie znaleziony. Tworzę tymczasowe dane...")
        return stworz_tymczasowe_dane()

def stworz_tymczasowe_dane():
    """Tworzy proste dane do testowania"""
    np.random.seed(42)
    X_train = np.random.rand(1000, 784)
    y_train = np.random.randint(0, 10, 1000)
    y_train_onehot = np.zeros((1000, 10))
    y_train_onehot[np.arange(1000), y_train] = 1
    
    X_test = np.random.rand(200, 784)
    y_test = np.random.randint(0, 10, 200)
    
    return (X_train, y_train), (X_test, y_test), (X_test, y_test)

def przygotuj_dane():
    """Przygotowuje dane do uczenia z podziałem na train/val/test"""
    train_set, valid_set, test_set = wczytaj_mnist()
    
    if len(train_set) == 2:  # Tymczasowe dane
        X_train, y_train = train_set
        X_val, y_val = valid_set
        X_test, y_test = test_set
    else:  # Prawdziwe MNIST
        X_train, y_train = train_set
        X_val, y_val = valid_set  
        X_test, y_test = test_set
    
    # Konwersja na one-hot tylko dla treningu
    y_train_onehot = np.zeros((y_train.size, 10))
    y_train_onehot[np.arange(y_train.size), y_train] = 1
    
    return (X_train, y_train_onehot, y_train), (X_val, y_val), (X_test, y_test)

# 4. EKSPERYMENTY Z DOKŁADNOŚCIĄ - POPRAWIONE
def eksperyment_rozmiary():
    """Eksperyment 1: Różne rozmiary pierwszej warstwy"""
    (X_train, y_train_onehot, y_train), (X_val, y_val), (X_test, y_test) = przygotuj_dane()
    
    # Użyj mniejszego podzbioru dla szybkości
    X_train_subset = X_train[:1000]
    y_train_subset = y_train_onehot[:1000]
    y_train_labels_subset = y_train[:1000]
    
    X_val_subset = X_val[:200]
    y_val_subset = y_val[:200]  # To są już etykiety, nie one-hot
    
    X_test_subset = X_test[:200]
    y_test_subset = y_test[:200]
    
    print(f"Train: {X_train_subset.shape}, Val: {X_val_subset.shape}")
    
    rozmiary = [32, 64, 128, 256]
    wyniki = {}
    
    for rozmiar in rozmiary:
        print(f"\n{'='*70}")
        print(f"TEST: Warstwa ukryta {rozmiar} neuronów")
        print(f"{'='*70}")
        
        siec = SiecNeuronowa()
        siec.dodaj_warstwe(Warstwa(784, rozmiar, 0.01))
        siec.dodaj_warstwe(WarstwaAktywacji(sigmoid, pochodna_sigmoid))
        siec.dodaj_warstwe(Warstwa(rozmiar, 10, 0.01))
        siec.dodaj_warstwe(WarstwaAktywacji(softmax, pochodna_softmax))
        
        # Trenuj z walidacją - POPRAWIONE: przekazujemy y_val_subset jako etykiety
        losses, accuracies = siec.fit(X_train_subset, y_train_subset, 
                                    X_val_subset, y_val_subset, epoki=30)
        
        # Testuj na danych testowych
        test_accuracy = siec.accuracy(X_test_subset, y_test_subset)
        
        wyniki[rozmiar] = {
            'final_loss': losses[-1],
            'final_train_acc': accuracies[-1],
            'test_accuracy': test_accuracy
        }
        
        print(f"⟹ Final - Train Acc: {accuracies[-1]:.4f}, Test Acc: {test_accuracy:.4f}")
    
    # PODSUMOWANIE
    print(f"\n{'='*70}")
    print("PODSUMOWANIE EKSPERYMENTU 1 - RÓŻNE ROZMIARY WARSTWY")
    print(f"{'='*70}")
    print("Rozmiar | Train Acc | Test Acc  | Final Loss")
    print("-" * 50)
    for rozmiar, metryki in wyniki.items():
        print(f"{rozmiar:7d} | {metryki['final_train_acc']:9.4f} | {metryki['test_accuracy']:9.4f} | {metryki['final_loss']:10.4f}")
    
    # Znajdź najlepszy rozmiar
    najlepszy = max(wyniki.items(), key=lambda x: x[1]['test_accuracy'])
    print(f"\n🏆 NAJLEPSZY: {najlepszy[0]} neuronów (Test Acc: {najlepszy[1]['test_accuracy']:.4f})")
    
    return wyniki

def trzy_warstwy():
    """Eksperyment 2: Dodanie trzeciej warstwy"""
    (X_train, y_train_onehot, y_train), (X_val, y_val), (X_test, y_test) = przygotuj_dane()
    
    X_train_subset = X_train[:1000]
    y_train_subset = y_train_onehot[:1000]
    
    X_val_subset = X_val[:200]
    y_val_subset = y_val[:200]  # Etykiety, nie one-hot
    
    X_test_subset = X_test[:200]
    y_test_subset = y_test[:200]
    
    print(f"\n{'='*70}")
    print("EKSPERYMENT 2: TRZY WARSTWY UKRYTE")
    print(f"{'='*70}")
    print("Architektura: 784 → 128 → 64 → 10")
    
    siec = SiecNeuronowa()
    siec.dodaj_warstwe(Warstwa(784, 128, 0.01))
    siec.dodaj_warstwe(WarstwaAktywacji(sigmoid, pochodna_sigmoid))
    siec.dodaj_warstwe(Warstwa(128, 64, 0.01))
    siec.dodaj_warstwe(WarstwaAktywacji(sigmoid, pochodna_sigmoid))
    siec.dodaj_warstwe(Warstwa(64, 10, 0.01))
    siec.dodaj_warstwe(WarstwaAktywacji(softmax, pochodna_softmax))
    
    losses, accuracies = siec.fit(X_train_subset, y_train_subset, 
                                X_val_subset, y_val_subset, epoki=30)
    
    test_accuracy = siec.accuracy(X_test_subset, y_test_subset)
    
    print(f"\n⟹ WYNIK: Train Acc: {accuracies[-1]:.4f}, Test Acc: {test_accuracy:.4f}")
    
    return test_accuracy

def pokaz_przyklady(siec, X_test, y_test, ile=5):
    """Pokazuje przykładowe predykcje"""
    print(f"\nPrzykładowe predykcje (pierwszych {ile}):")
    print("True | Pred | ✓/✗")
    print("-" * 15)
    
    predictions = siec.predict(X_test[:ile])
    predicted_labels = np.argmax(predictions, axis=1)
    
    for i in range(ile):
        true_label = y_test[i]
        pred_label = predicted_labels[i]
        correct = "✓" if true_label == pred_label else "✗"
        print(f"{true_label:4} | {pred_label:4} | {correct}")

# 5. GŁÓWNY PROGRAM
def main():
    print("SIECI NEURONOWE - EKSPERYMENTY Z DOKŁADNOŚCIĄ")
    print("=" * 70)
    
    (X_train, y_train_onehot, y_train), (X_val, y_val), (X_test, y_test) = przygotuj_dane()
    print(f"Dane: Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")
    print(f"Etykiety - Train: {y_train.shape}, Val: {y_val.shape}, Test: {y_test.shape}")
    
    while True:
        print("\n" + "="*50)
        print("MENU GŁÓWNE")
        print("="*50)
        print("1 - Różne rozmiary warstwy (z dokładnością)")
        print("2 - Trzy warstwy ukryte (z dokładnością)") 
        print("3 - Oba eksperymenty")
        print("4 - Pokaż przykładowe predykcje")
        print("0 - Wyjście")
        print("="*50)
        
        try:
            wybor = input("Twój wybór (0-4): ").strip()
            
            if wybor == '1':
                eksperyment_rozmiary()
            elif wybor == '2':
                trzy_warstwy()
            elif wybor == '3':
                print("\n" + "="*70)
                print("URUCHAMIANIE OBIU EKSPERYMENTÓW")
                print("="*70)
                wyniki1 = eksperyment_rozmiary()
                wyniki2 = trzy_warstwy()
            elif wybor == '4':
                # Prosta sieć do pokazania przykładów
                siec = SiecNeuronowa()
                siec.dodaj_warstwe(Warstwa(784, 64, 0.01))
                siec.dodaj_warstwe(WarstwaAktywacji(sigmoid, pochodna_sigmoid))
                siec.dodaj_warstwe(Warstwa(64, 10, 0.01))
                siec.dodaj_warstwe(WarstwaAktywacji(softmax, pochodna_softmax))
                
                # Wytrenuj szybko na małych danych
                X_temp = X_train[:100]
                y_temp = np.zeros((100, 10))
                y_temp[np.arange(100), y_train[:100]] = 1
                siec.fit(X_temp, y_temp, epoki=5)
                
                pokaz_przyklady(siec, X_test, y_test, 10)
            elif wybor == '0':
                print("\nDo widzenia!")
                break
            else:
                print("Nieprawidłowy wybór.")
                
            if wybor != '0':
                input("\nNaciśnij Enter aby kontynuować...")
                
        except KeyboardInterrupt:
            print("\nPrzerwano. Do widzenia!")
            break

if __name__ == "__main__":
    main()

SIECI NEURONOWE - EKSPERYMENTY Z DOKŁADNOŚCIĄ
Dane: Train: (50000, 784), Val: (10000, 784), Test: (10000, 784)
Etykiety - Train: (50000,), Val: (10000,), Test: (10000,)

MENU GŁÓWNE
1 - Różne rozmiary warstwy (z dokładnością)
2 - Trzy warstwy ukryte (z dokładnością)
3 - Oba eksperymenty
4 - Pokaż przykładowe predykcje
0 - Wyjście
Train: (1000, 784), Val: (200, 784)

TEST: Warstwa ukryta 32 neuronów
Epoka   0, Loss: 0.0904, Train Accuracy: 0.1080, Val Accuracy: 0.1000
Epoka  10, Loss: 0.0790, Train Accuracy: 0.3140, Val Accuracy: 0.4150
Epoka  20, Loss: 0.0695, Train Accuracy: 0.4150, Val Accuracy: 0.3700
⟹ Final - Train Acc: 0.4890, Test Acc: 0.3750

TEST: Warstwa ukryta 64 neuronów
Epoka   0, Loss: 0.0911, Train Accuracy: 0.1270, Val Accuracy: 0.0900
Epoka  10, Loss: 0.0825, Train Accuracy: 0.2410, Val Accuracy: 0.1650
Epoka  20, Loss: 0.0892, Train Accuracy: 0.1300, Val Accuracy: 0.2300
⟹ Final - Train Acc: 0.3500, Test Acc: 0.3550

TEST: Warstwa ukryta 128 neuronów
Epoka   0, Loss: 

In [23]:
import numpy as np
import pickle
import gzip

class Warstwa:
    def __init__(self, rozmiar_wejscia, rozmiar_wyjscia, learning_rate=0.01):
        # Inicjalizacja wag - uwzględniamy bias jako pierwszą kolumnę
        self.rozmiar_wejscia = rozmiar_wejscia
        self.rozmiar_wyjscia = rozmiar_wyjscia
        # Wagi: rozmiar_wyjscia × (rozmiar_wejscia + 1) - +1 dla biasu
        self.W = np.random.randn(rozmiar_wyjscia, rozmiar_wejscia + 1) * 0.1
        self.learning_rate = learning_rate
        
        # Pamiętanie danych do propagacji wstecznej
        self.x = None  # wektor wejściowy z dodaną jedynką
        self.net = None  # net = W · x
        self.a = None   # a = φ(net)
    
    def forward(self, a_prev):
        """Propagacja w przód - zgodnie z opisem matematycznym"""
        # a_prev to wektor z poprzedniej warstwy (bez biasu)
        # Tworzymy x = [1, a_prevᵀ]ᵀ
        self.x = np.hstack([np.ones((a_prev.shape[0], 1)), a_prev])
        
        # net = W · x
        self.net = np.dot(self.x, self.W.T)  # W · x
        
        # a = φ(net)
        self.a = self._funkcja_aktywacji(self.net)
        
        return self.a
    
    def backward(self, delta_nastepna, W_nastepna=None):
        """Propagacja w tył - zgodnie z opisem matematycznym"""
        if W_nastepna is not None:
            # Warstwa ukryta: δ[l] = (W[l+1]ᵀ · δ[l+1]) ⊙ φ'(net[l])
            # Pomijamy bias w wagach następnej warstwy
            W_nastepna_bez_biasu = W_nastepna[:, 1:]  # pomijamy wagę dla biasu
            delta = np.dot(delta_nastepna, W_nastepna_bez_biasu) * self._pochodna_aktywacji(self.net)
        else:
            # Warstwa wyjściowa: δ[L] = (a[L] - y) ⊙ φ'(net[L])
            delta = delta_nastepna * self._pochodna_aktywacji(self.net)
        
        # Obliczanie gradientu: ∂L/∂W = δ · xᵀ
        gradient_W = np.dot(delta.T, self.x)
        
        # Aktualizacja wag: W = W - η · ∂L/∂W
        self.W -= self.learning_rate * gradient_W
        
        return delta, self.W
    
    def _funkcja_aktywacji(self, x):
        """Funkcja aktywacji - sigmoid"""
        return 1 / (1 + np.exp(-np.clip(x, -250, 250)))  # clip dla stabilności
    
    def _pochodna_aktywacji(self, x):
        """Pochodna funkcji aktywacji"""
        s = self._funkcja_aktywacji(x)
        return s * (1 - s)

class SiecNeuronowa:
    def __init__(self):
        self.warstwy = []
    
    def dodaj_warstwe(self, warstwa):
        self.warstwy.append(warstwa)
    
    def forward(self, X):
        """Propagacja w przód przez wszystkie warstwy"""
        a = X
        for warstwa in self.warstwy:
            a = warstwa.forward(a)
        return a
    
    def backward(self, y):
        """Propagacja w tył - zgodnie z opisem matematycznym"""
        # Ostatnia warstwa: δ[L] = (a[L] - y) ⊙ φ'(net[L])
        ostatnia_warstwa = self.warstwy[-1]
        delta_L = (ostatnia_warstwa.a - y) * ostatnia_warstwa._pochodna_aktywacji(ostatnia_warstwa.net)
        
        # Propagacja wstecz przez warstwy
        delta = delta_L
        W_nastepna = None
        
        for i in range(len(self.warstwy) - 1, 0, -1):
            delta, W_nastepna = self.warstwy[i].backward(delta, W_nastepna)
        
        # Pierwsza warstwa ukryta
        self.warstwy[0].backward(delta, W_nastepna)
    
    def fit(self, X, y, epoki=10, verbose=True):
        """Uczenie sieci"""
        losses = []
        accuracies = []
        
        for epoka in range(epoki):
            # Propagacja w przód
            output = self.forward(X)
            
            # Obliczanie funkcji kosztu: L = ½∑(y - a[L])²
            loss = 0.5 * np.sum((output - y) ** 2) / X.shape[0]
            losses.append(loss)
            
            # Obliczanie dokładności
            predictions = np.argmax(output, axis=1)
            true_labels = np.argmax(y, axis=1)
            accuracy = np.mean(predictions == true_labels)
            accuracies.append(accuracy)
            
            # Propagacja w tył
            self.backward(y)
            
            if verbose and epoka % 10 == 0:
                print(f"Epoka {epoka:3d}, Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")
        
        return losses, accuracies
    
    def predict(self, X):
        """Predykcja dla nowych danych"""
        return self.forward(X)
    
    def accuracy(self, X, y_true):
        """Oblicza dokładność"""
        predictions = self.predict(X)
        predicted_labels = np.argmax(predictions, axis=1)
        return np.mean(predicted_labels == y_true)

# Funkcje do wczytywania danych (bez zmian)
def wczytaj_mnist(sciezka='mnist.pkl.gz'):
    try:
        with gzip.open(sciezka, 'rb') as f:
            train_set, valid_set, test_set = pickle.load(f, encoding='latin1')
        return train_set, valid_set, test_set
    except FileNotFoundError:
        print("Plik MNIST nie znaleziony. Tworzę tymczasowe dane...")
        return stworz_tymczasowe_dane()

def stworz_tymczasowe_dane():
    np.random.seed(42)
    X_train = np.random.rand(1000, 784)
    y_train = np.random.randint(0, 10, 1000)
    y_train_onehot = np.zeros((1000, 10))
    y_train_onehot[np.arange(1000), y_train] = 1
    
    X_test = np.random.rand(200, 784)
    y_test = np.random.randint(0, 10, 200)
    
    return (X_train, y_train), (X_test, y_test), (X_test, y_test)

def przygotuj_dane():
    train_set, valid_set, test_set = wczytaj_mnist()
    
    if len(train_set) == 2:
        X_train, y_train = train_set
        X_val, y_val = valid_set
        X_test, y_test = test_set
    else:
        X_train, y_train = train_set
        X_val, y_val = valid_set  
        X_test, y_test = test_set
    
    y_train_onehot = np.zeros((y_train.size, 10))
    y_train_onehot[np.arange(y_train.size), y_train] = 1
    
    return (X_train, y_train_onehot, y_train), (X_val, y_val), (X_test, y_test)

# Eksperymenty
def eksperyment_rozmiary():
    (X_train, y_train_onehot, y_train), (X_val, y_val), (X_test, y_test) = przygotuj_dane()
    
    X_train_subset = X_train[:1000]
    y_train_subset = y_train_onehot[:1000]
    
    X_val_subset = X_val[:200]
    y_val_subset = y_val[:200]
    
    X_test_subset = X_test[:200]
    y_test_subset = y_test[:200]
    
    print(f"Train: {X_train_subset.shape}, Val: {X_val_subset.shape}")
    
    rozmiary = [32, 64, 128, 256]
    wyniki = {}
    
    for rozmiar in rozmiary:
        print(f"\n{'='*70}")
        print(f"TEST: Warstwa ukryta {rozmiar} neuronów")
        print(f"{'='*70}")
        
        siec = SiecNeuronowa()
        siec.dodaj_warstwe(Warstwa(784, rozmiar, 0.1))  # wyższy learning_rate
        siec.dodaj_warstwe(Warstwa(rozmiar, 10, 0.1))
        
        losses, accuracies = siec.fit(X_train_subset, y_train_subset, epoki=50, verbose=False)
        
        # Wypisz postęp co 10 epok
        for epoka in range(0, 50, 10):
            if epoka < len(losses):
                print(f"Epoka {epoka:3d}, Loss: {losses[epoka]:.4f}, Accuracy: {accuracies[epoka]:.4f}")
        
        test_accuracy = siec.accuracy(X_test_subset, y_test_subset)
        
        wyniki[rozmiar] = {
            'final_loss': losses[-1],
            'final_train_acc': accuracies[-1],
            'test_accuracy': test_accuracy
        }
        
        print(f"⟹ Final - Train Acc: {accuracies[-1]:.4f}, Test Acc: {test_accuracy:.4f}")
    
    # Podsumowanie
    print(f"\n{'='*70}")
    print("PODSUMOWANIE - RÓŻNE ROZMIARY WARSTWY")
    print(f"{'='*70}")
    print("Rozmiar | Train Acc | Test Acc  | Final Loss")
    print("-" * 50)
    for rozmiar, metryki in wyniki.items():
        print(f"{rozmiar:7d} | {metryki['final_train_acc']:9.4f} | {metryki['test_accuracy']:9.4f} | {metryki['final_loss']:10.4f}")
    
    najlepszy = max(wyniki.items(), key=lambda x: x[1]['test_accuracy'])
    print(f"\n🏆 NAJLEPSZY: {najlepszy[0]} neuronów (Test Acc: {najlepszy[1]['test_accuracy']:.4f})")
    
    return wyniki

def trzy_warstwy():
    (X_train, y_train_onehot, y_train), (X_val, y_val), (X_test, y_test) = przygotuj_dane()
    
    X_train_subset = X_train[:1000]
    y_train_subset = y_train_onehot[:1000]
    
    X_test_subset = X_test[:200]
    y_test_subset = y_test[:200]
    
    print(f"\n{'='*70}")
    print("EKSPERYMENT: TRZY WARSTWY UKRYTE")
    print(f"{'='*70}")
    print("Architektura: 784 → 128 → 64 → 10")
    
    siec = SiecNeuronowa()
    siec.dodaj_warstwe(Warstwa(784, 128, 0.1))
    siec.dodaj_warstwe(Warstwa(128, 64, 0.1))
    siec.dodaj_warstwe(Warstwa(64, 10, 0.1))
    
    losses, accuracies = siec.fit(X_train_subset, y_train_subset, epoki=50, verbose=False)
    
    # Wypisz postęp
    for epoka in range(0, 50, 10):
        if epoka < len(losses):
            print(f"Epoka {epoka:3d}, Loss: {losses[epoka]:.4f}, Accuracy: {accuracies[epoka]:.4f}")
    
    test_accuracy = siec.accuracy(X_test_subset, y_test_subset)
    print(f"⟹ Final - Train Acc: {accuracies[-1]:.4f}, Test Acc: {test_accuracy:.4f}")
    
    return test_accuracy

# Główny program
def main():
    print("SIECI NEURONOWE - IMPLEMENTACJA WG OPISU MATEMATYCZNEGO")
    print("=" * 70)
    
    (X_train, y_train_onehot, y_train), (X_val, y_val), (X_test, y_test) = przygotuj_dane()
    print(f"Dane: Train: {X_train.shape}, Test: {X_test.shape}")
    
    while True:
        print("\n" + "="*50)
        print("MENU GŁÓWNE")
        print("="*50)
        print("1 - Różne rozmiary warstwy ukrytej")
        print("2 - Trzy warstwy ukryte") 
        print("0 - Wyjście")
        print("="*50)
        
        wybor = input("Twój wybór (0-2): ").strip()
        
        if wybor == '1':
            eksperyment_rozmiary()
        elif wybor == '2':
            trzy_warstwy()
        elif wybor == '0':
            print("\nDo widzenia!")
            break
        else:
            print("Nieprawidłowy wybór.")
        
        if wybor != '0':
            input("\nNaciśnij Enter aby kontynuować...")

if __name__ == "__main__":
    main()

SIECI NEURONOWE - IMPLEMENTACJA WG OPISU MATEMATYCZNEGO
Dane: Train: (50000, 784), Test: (10000, 784)

MENU GŁÓWNE
1 - Różne rozmiary warstwy ukrytej
2 - Trzy warstwy ukryte
0 - Wyjście
Train: (1000, 784), Val: (200, 784)

TEST: Warstwa ukryta 32 neuronów
Epoka   0, Loss: 1.3257, Accuracy: 0.1470
Epoka  10, Loss: 0.5000, Accuracy: 0.1050
Epoka  20, Loss: 0.5000, Accuracy: 0.1050
Epoka  30, Loss: 0.5000, Accuracy: 0.1050
Epoka  40, Loss: 0.5000, Accuracy: 0.1050
⟹ Final - Train Acc: 0.1050, Test Acc: 0.1400

TEST: Warstwa ukryta 64 neuronów
Epoka   0, Loss: 1.2820, Accuracy: 0.1130
Epoka  10, Loss: 0.5000, Accuracy: 0.1160
Epoka  20, Loss: 0.5000, Accuracy: 0.1160
Epoka  30, Loss: 0.5000, Accuracy: 0.1160
Epoka  40, Loss: 0.5000, Accuracy: 0.1160
⟹ Final - Train Acc: 0.1160, Test Acc: 0.1400

TEST: Warstwa ukryta 128 neuronów
Epoka   0, Loss: 1.4731, Accuracy: 0.0740
Epoka  10, Loss: 0.5000, Accuracy: 0.1170
Epoka  20, Loss: 0.5000, Accuracy: 0.1170
Epoka  30, Loss: 0.5000, Accuracy: 0.

In [24]:
import numpy as np
import pickle
import gzip

class Warstwa:
    def __init__(self, rozmiar_wejscia, rozmiar_wyjscia, learning_rate=0.01):
        self.rozmiar_wejscia = rozmiar_wejscia
        self.rozmiar_wyjscia = rozmiar_wyjscia
        self.learning_rate = learning_rate
        
        # INICJALIZACJA WAG Z ROZKŁADU NORMALNEGO N(0, 1/liczba_neuronow)
        # Wagi: rozmiar_wyjscia × (rozmiar_wejscia + 1) - +1 dla biasu
        std_dev = 1.0 / np.sqrt(rozmiar_wejscia)  # N(0, 1/n[l-1])
        self.W = np.random.randn(rozmiar_wyjscia, rozmiar_wejscia + 1) * std_dev
        
        # Pamiętanie danych do propagacji wstecznej
        self.x = None  # wektor wejściowy z dodaną jedynką
        self.net = None  # net = W · x
        self.a = None   # a = φ(net)
    
    def forward(self, a_prev):
        """Propagacja w przód - zgodnie z opisem matematycznym"""
        # a_prev to wektor z poprzedniej warstwy (bez biasu)
        # Tworzymy x = [1, a_prevᵀ]ᵀ
        self.x = np.hstack([np.ones((a_prev.shape[0], 1)), a_prev])
        
        # net = W · x
        self.net = np.dot(self.x, self.W.T)  # W · x
        
        # a = φ(net)
        self.a = self._funkcja_aktywacji(self.net)
        
        return self.a
    
    def backward(self, delta_nastepna, W_nastepna=None):
        """Propagacja w tył - zgodnie z opisem matematycznym"""
        if W_nastepna is not None:
            # Warstwa ukryta: δ[l] = (W[l+1]ᵀ · δ[l+1]) ⊙ φ'(net[l])
            # Pomijamy bias w wagach następnej warstwy
            W_nastepna_bez_biasu = W_nastepna[:, 1:]  # pomijamy wagę dla biasu
            delta = np.dot(delta_nastepna, W_nastepna_bez_biasu) * self._pochodna_aktywacji(self.net)
        else:
            # Warstwa wyjściowa: δ[L] = (a[L] - y) ⊙ φ'(net[L])
            delta = delta_nastepna * self._pochodna_aktywacji(self.net)
        
        # Obliczanie gradientu: ∂L/∂W = δ · xᵀ
        gradient_W = np.dot(delta.T, self.x)
        
        # Aktualizacja wag: W = W - η · ∂L/∂W
        self.W -= self.learning_rate * gradient_W
        
        return delta, self.W
    
    def _funkcja_aktywacji(self, x):
        """Funkcja aktywacji - sigmoid"""
        return 1 / (1 + np.exp(-np.clip(x, -250, 250)))  # clip dla stabilności
    
    def _pochodna_aktywacji(self, x):
        """Pochodna funkcji aktywacji"""
        s = self._funkcja_aktywacji(x)
        return s * (1 - s)

class SiecNeuronowa:
    def __init__(self):
        self.warstwy = []
    
    def dodaj_warstwe(self, warstwa):
        self.warstwy.append(warstwa)
    
    def forward(self, X):
        """Propagacja w przód przez wszystkie warstwy"""
        a = X
        for warstwa in self.warstwy:
            a = warstwa.forward(a)
        return a
    
    def backward(self, y):
        """Propagacja w tył - zgodnie z opisem matematycznym"""
        # Ostatnia warstwa: δ[L] = (a[L] - y) ⊙ φ'(net[L])
        ostatnia_warstwa = self.warstwy[-1]
        delta_L = (ostatnia_warstwa.a - y) * ostatnia_warstwa._pochodna_aktywacji(ostatnia_warstwa.net)
        
        # Propagacja wstecz przez warstwy
        delta = delta_L
        W_nastepna = None
        
        for i in range(len(self.warstwy) - 1, 0, -1):
            delta, W_nastepna = self.warstwy[i].backward(delta, W_nastepna)
        
        # Pierwsza warstwa ukryta
        self.warstwy[0].backward(delta, W_nastepna)
    
    def fit(self, X, y, epoki=10, verbose=True):
        """Uczenie sieci"""
        losses = []
        accuracies = []
        
        for epoka in range(epoki):
            # Propagacja w przód
            output = self.forward(X)
            
            # Obliczanie funkcji kosztu: L = ½∑(y - a[L])²
            loss = 0.5 * np.sum((output - y) ** 2) / X.shape[0]
            losses.append(loss)
            
            # Obliczanie dokładności
            predictions = np.argmax(output, axis=1)
            true_labels = np.argmax(y, axis=1)
            accuracy = np.mean(predictions == true_labels)
            accuracies.append(accuracy)
            
            # Propagacja w tył
            self.backward(y)
            
            if verbose and epoka % 10 == 0:
                print(f"Epoka {epoka:3d}, Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")
        
        return losses, accuracies
    
    def predict(self, X):
        """Predykcja dla nowych danych"""
        return self.forward(X)
    
    def accuracy(self, X, y_true):
        """Oblicza dokładność"""
        predictions = self.predict(X)
        predicted_labels = np.argmax(predictions, axis=1)
        return np.mean(predicted_labels == y_true)

# Funkcje do wczytywania danych
def wczytaj_mnist(sciezka='mnist.pkl.gz'):
    try:
        with gzip.open(sciezka, 'rb') as f:
            train_set, valid_set, test_set = pickle.load(f, encoding='latin1')
        return train_set, valid_set, test_set
    except FileNotFoundError:
        print("Plik MNIST nie znaleziony. Tworzę tymczasowe dane...")
        return stworz_tymczasowe_dane()

def stworz_tymczasowe_dane():
    np.random.seed(42)
    X_train = np.random.rand(1000, 784)
    y_train = np.random.randint(0, 10, 1000)
    y_train_onehot = np.zeros((1000, 10))
    y_train_onehot[np.arange(1000), y_train] = 1
    
    X_test = np.random.rand(200, 784)
    y_test = np.random.randint(0, 10, 200)
    
    return (X_train, y_train), (X_test, y_test), (X_test, y_test)

def przygotuj_dane():
    train_set, valid_set, test_set = wczytaj_mnist()
    
    if len(train_set) == 2:
        X_train, y_train = train_set
        X_val, y_val = valid_set
        X_test, y_test = test_set
    else:
        X_train, y_train = train_set
        X_val, y_val = valid_set  
        X_test, y_test = test_set
    
    y_train_onehot = np.zeros((y_train.size, 10))
    y_train_onehot[np.arange(y_train.size), y_train] = 1
    
    return (X_train, y_train_onehot, y_train), (X_val, y_val), (X_test, y_test)

# Eksperymenty z różnymi inicjalizacjami
def test_inicjalizacji():
    """Test różnych metod inicjalizacji wag"""
    print("TEST INICJALIZACJI WAG")
    print("=" * 70)
    
    rozmiary = [784, 128, 64, 10]
    
    print("\n1. INICJALIZACJA N(0, 0.1) - STARA METODA:")
    for i in range(len(rozmiary)-1):
        rozmiar_wejscia = rozmiary[i]
        rozmiar_wyjscia = rozmiary[i+1]
        wagi = np.random.randn(rozmiar_wyjscia, rozmiar_wejscia + 1) * 0.1
        srednia = np.mean(wagi)
        std = np.std(wagi)
        print(f"Warstwa {i+1}: {rozmiar_wejscia}→{rozmiar_wyjscia}, Średnia: {srednia:.4f}, Std: {std:.4f}")
    
    print("\n2. INICJALIZACJA N(0, 1/√n[l-1]) - NOWA METODA:")
    for i in range(len(rozmiary)-1):
        rozmiar_wejscia = rozmiary[i]
        rozmiar_wyjscia = rozmiary[i+1]
        std_dev = 1.0 / np.sqrt(rozmiar_wejscia)
        wagi = np.random.randn(rozmiar_wyjscia, rozmiar_wejscia + 1) * std_dev
        srednia = np.mean(wagi)
        std = np.std(wagi)
        print(f"Warstwa {i+1}: {rozmiar_wejscia}→{rozmiar_wyjscia}, Średnia: {srednia:.4f}, Std: {std:.4f} (1/√{rozmiar_wejscia} = {std_dev:.4f})")

def eksperyment_rozmiary():
    (X_train, y_train_onehot, y_train), (X_val, y_val), (X_test, y_test) = przygotuj_dane()
    
    X_train_subset = X_train[:1000]
    y_train_subset = y_train_onehot[:1000]
    
    X_val_subset = X_val[:200]
    y_val_subset = y_val[:200]
    
    X_test_subset = X_test[:200]
    y_test_subset = y_test[:200]
    
    print(f"Train: {X_train_subset.shape}, Val: {X_val_subset.shape}")
    
    rozmiary = [32, 64, 128, 256]
    wyniki = {}
    
    for rozmiar in rozmiary:
        print(f"\n{'='*70}")
        print(f"TEST: Warstwa ukryta {rozmiar} neuronów (inicjalizacja N(0, 1/√n))")
        print(f"{'='*70}")
        
        siec = SiecNeuronowa()
        siec.dodaj_warstwe(Warstwa(784, rozmiar, 0.1))
        siec.dodaj_warstwe(Warstwa(rozmiar, 10, 0.1))
        
        losses, accuracies = siec.fit(X_train_subset, y_train_subset, epoki=50, verbose=False)
        
        # Wypisz postęp co 10 epok
        for epoka in range(0, 50, 10):
            if epoka < len(losses):
                print(f"Epoka {epoka:3d}, Loss: {losses[epoka]:.4f}, Accuracy: {accuracies[epoka]:.4f}")
        
        test_accuracy = siec.accuracy(X_test_subset, y_test_subset)
        
        wyniki[rozmiar] = {
            'final_loss': losses[-1],
            'final_train_acc': accuracies[-1],
            'test_accuracy': test_accuracy
        }
        
        print(f"⟹ Final - Train Acc: {accuracies[-1]:.4f}, Test Acc: {test_accuracy:.4f}")
    
    # Podsumowanie
    print(f"\n{'='*70}")
    print("PODSUMOWANIE - RÓŻNE ROZMIARY WARSTWY")
    print(f"{'='*70}")
    print("Rozmiar | Train Acc | Test Acc  | Final Loss")
    print("-" * 50)
    for rozmiar, metryki in wyniki.items():
        print(f"{rozmiar:7d} | {metryki['final_train_acc']:9.4f} | {metryki['test_accuracy']:9.4f} | {metryki['final_loss']:10.4f}")
    
    najlepszy = max(wyniki.items(), key=lambda x: x[1]['test_accuracy'])
    print(f"\n🏆 NAJLEPSZY: {najlepszy[0]} neuronów (Test Acc: {najlepszy[1]['test_accuracy']:.4f})")
    
    return wyniki

def trzy_warstwy():
    (X_train, y_train_onehot, y_train), (X_val, y_val), (X_test, y_test) = przygotuj_dane()
    
    X_train_subset = X_train[:1000]
    y_train_subset = y_train_onehot[:1000]
    
    X_test_subset = X_test[:200]
    y_test_subset = y_test[:200]
    
    print(f"\n{'='*70}")
    print("EKSPERYMENT: TRZY WARSTWY UKRYTE (inicjalizacja N(0, 1/√n))")
    print(f"{'='*70}")
    print("Architektura: 784 → 128 → 64 → 10")
    
    siec = SiecNeuronowa()
    siec.dodaj_warstwe(Warstwa(784, 128, 0.1))
    siec.dodaj_warstwe(Warstwa(128, 64, 0.1))
    siec.dodaj_warstwe(Warstwa(64, 10, 0.1))
    
    losses, accuracies = siec.fit(X_train_subset, y_train_subset, epoki=50, verbose=False)
    
    # Wypisz postęp
    for epoka in range(0, 50, 10):
        if epoka < len(losses):
            print(f"Epoka {epoka:3d}, Loss: {losses[epoka]:.4f}, Accuracy: {accuracies[epoka]:.4f}")
    
    test_accuracy = siec.accuracy(X_test_subset, y_test_subset)
    print(f"⟹ Final - Train Acc: {accuracies[-1]:.4f}, Test Acc: {test_accuracy:.4f}")
    
    return test_accuracy

# Główny program
def main():
    print("SIECI NEURONOWE - POPRAWNA INICJALIZACJA WAG N(0, 1/√n)")
    print("=" * 70)
    
    # Test różnych inicjalizacji
    test_inicjalizacji()
    
    (X_train, y_train_onehot, y_train), (X_val, y_val), (X_test, y_test) = przygotuj_dane()
    print(f"\nDane: Train: {X_train.shape}, Test: {X_test.shape}")
    
    while True:
        print("\n" + "="*50)
        print("MENU GŁÓWNE")
        print("="*50)
        print("1 - Różne rozmiary warstwy ukrytej")
        print("2 - Trzy warstwy ukryte") 
        print("3 - Test szybki (małe dane)")
        print("0 - Wyjście")
        print("="*50)
        
        wybor = input("Twój wybór (0-3): ").strip()
        
        if wybor == '1':
            eksperyment_rozmiary()
        elif wybor == '2':
            trzy_warstwy()
        elif wybor == '3':
            # Szybki test
            X_temp = np.random.rand(50, 784)
            y_temp = np.zeros((50, 10))
            y_labels = np.random.randint(0, 10, 50)
            y_temp[np.arange(50), y_labels] = 1
            
            siec = SiecNeuronowa()
            siec.dodaj_warstwe(Warstwa(784, 32, 0.1))
            siec.dodaj_warstwe(Warstwa(32, 10, 0.1))
            siec.fit(X_temp, y_temp, epoki=10, verbose=True)
        elif wybor == '0':
            print("\nDo widzenia!")
            break
        else:
            print("Nieprawidłowy wybór.")
        
        if wybor != '0':
            input("\nNaciśnij Enter aby kontynuować...")

if __name__ == "__main__":
    main()

SIECI NEURONOWE - POPRAWNA INICJALIZACJA WAG N(0, 1/√n)
TEST INICJALIZACJI WAG

1. INICJALIZACJA N(0, 0.1) - STARA METODA:
Warstwa 1: 784→128, Średnia: -0.0004, Std: 0.0999
Warstwa 2: 128→64, Średnia: -0.0007, Std: 0.1009
Warstwa 3: 64→10, Średnia: -0.0005, Std: 0.1003

2. INICJALIZACJA N(0, 1/√n[l-1]) - NOWA METODA:
Warstwa 1: 784→128, Średnia: -0.0000, Std: 0.0357 (1/√784 = 0.0357)
Warstwa 2: 128→64, Średnia: -0.0002, Std: 0.0880 (1/√128 = 0.0884)
Warstwa 3: 64→10, Średnia: -0.0011, Std: 0.1311 (1/√64 = 0.1250)

Dane: Train: (50000, 784), Test: (10000, 784)

MENU GŁÓWNE
1 - Różne rozmiary warstwy ukrytej
2 - Trzy warstwy ukryte
3 - Test szybki (małe dane)
0 - Wyjście
Train: (1000, 784), Val: (200, 784)

TEST: Warstwa ukryta 32 neuronów (inicjalizacja N(0, 1/√n))
Epoka   0, Loss: 1.3432, Accuracy: 0.0990
Epoka  10, Loss: 0.5000, Accuracy: 0.0930
Epoka  20, Loss: 0.5000, Accuracy: 0.0930
Epoka  30, Loss: 0.5000, Accuracy: 0.0930
Epoka  40, Loss: 0.5000, Accuracy: 0.0930
⟹ Final - Train